# Analyze Unlabel from Re-label 1

1. Import data from Relabel_result.xlsx
2. Select data for each observation that are unlabeled
3. Find patterns using RegEx and N-grams (use function from Analyze_group_c.ipynb)
4. Find word before and after keyword
5. Add the result to negative, positive and uncertain key in Relabel_group_c.ipynb
6. Re-label again in Relabel_group_c.ipynb

## Import Data

In [388]:
# from google.colab import drive

# drive.mount('/content/drive')

In [389]:
# %cd /content/drive/My Drive/Pre-WiL

In [390]:
import pandas as pd

cardio_data = pd.read_excel('Relabel_result.xlsx', sheet_name='Cardiomegaly')
pleural_data = pd.read_excel('Relabel_result.xlsx', sheet_name='Pleural Effusion')
atelec_data = pd.read_excel('Relabel_result.xlsx', sheet_name='Atelectasis')
lesion_data = pd.read_excel('Relabel_result.xlsx', sheet_name='Lung Lesion')
v1_data = pd.read_excel('Relabel_result.xlsx', sheet_name='Inspectra Lung Opacity v1')
edema_data = pd.read_excel('Relabel_result.xlsx', sheet_name='Edema')

## Function

In [391]:
# create dataframe of each observations
def selectData(df, name, value):
  # choose rows
  if value == None:
    name_df = df[df[name + ' Ground Truth'].isna()]
  else:
    name_df = df[df[name + ' Ground Truth'] == value]

  return name_df

In [392]:
# create list of reports
import re
from ast import literal_eval

def createReportList(df):
  return list(map(literal_eval, df['Report List']))

In [393]:
# create n-grams
from collections import defaultdict
import re
from nltk import ngrams

def createPatternNgrams(data, n, keyword=None):
  pattern_dict = defaultdict(lambda: 0)
  for report in data:
    for sent in report[1:]:
      if (keyword != None and re.search(keyword, sent)) or keyword == None:
        for word in ngrams(sent.split(), n):
          pattern_dict[word] += 1    

  pattern_dict = sorted(pattern_dict.items(), key=lambda x: x[1], reverse=True)

  return dict(pattern_dict)

In [394]:
# find word before input word
def findWordBefore(bigrams_dict, word):
  word_before = defaultdict(lambda: 0)
  for k,v in bigrams_dict.items():
    if re.search(word, k[1]):
      word_before[k[0]] += v

  word_before = sorted(word_before.items(), key=lambda x: x[1], reverse=True)

  return dict(word_before)

def findWordBefore2(trigrams_dict, word):
  word_before = defaultdict(lambda: 0)
  for k,v in trigrams_dict.items():
    if re.search(word, k[2]):
      word_before[k[0]] += v

  word_before = sorted(word_before.items(), key=lambda x: x[1], reverse=True)

  return dict(word_before)

In [395]:
# find word after v.to be
def findWordAfter(trigrams_dict, word):
  word_after = defaultdict(lambda: 0)
  for k,v in trigrams_dict.items():
    if re.search(word, k[0]) and re.search(r'is|are|was|were', k[1]):
      word_after[k[2]] += v

  word_after = sorted(word_after.items(), key=lambda x: x[1], reverse=True)

  return dict(word_after)

# Cardiomegaly

In [396]:
# define keyword
cardio_keyword = r'card|heart|lv|ventricular enlarge'

## Ground truth 0

Prepare Data

In [397]:
# choose reports
cardio_df_0 = selectData(cardio_data, 'Cardiomegaly', 0.0)
cardio_df_0

,Reports,Report List,Cardiomegaly BERT Labeler,Cardiomegaly Inspectra Labeler,Cardiomegaly Ground Truth,Cardiomegaly Key Detect
1,CXR PA upright\n The chest shows round calcif...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,0.0,"<re.Match object; span=(8, 21), match='heart i..."
10,CHEST\n\nMinimal fibrotic change at LUL.\nThe ...,"['<s> chest <\\s>', '<s> minimal fibrotic chan...",0,1,0.0,0.5
14,CHEST.;\nLarge soft tissue mass at right parat...,"['<s> chest <\\s>', '<s> large soft tissue mas...",1,0,0.0,"<re.Match object; span=(4, 53), match='no card..."
16,CHEST PA UPRIGHT\n\nhistory of ca lung post ri...,"['<s> chest pa upright <\\s>', '<s> history of...",1,0,0.0,"<re.Match object; span=(4, 29), match='no card..."
19,CXR \n\nHistory of hemoptysis\n\nThere is mass...,"['<s> cxr <\\s>', '<s> history of hemoptysis <...",1,0,0.0,"<re.Match object; span=(13, 33), match='no car..."
...,...,...,...,...,...,...
39236,CXR PA UPRIGHT\n\nFINDINGS : No detectable pu...,"['<s> cxr pa upright <\\s>', '<s> findings no ...",1,0,0.0,"<re.Match object; span=(4, 32), match='no card..."
39237,CXR PA UPRIGHT\n\nFINDINGS : No detectable pu...,"['<s> cxr pa upright <\\s>', '<s> findings no ...",1,0,0.0,"<re.Match object; span=(4, 32), match='no card..."
39238,CXR(PA)\n\n IMP:\n No active pulmonary infi...,"['<s> cxr pa <\\s>', '<s> imp <\\s>', '<s> no ...",1,0,0.0,"<re.Match object; span=(13, 33), match='no car..."
39239,CXR(PA)\n\n IMP:\n No active pulmonary infi...,"['<s> cxr pa <\\s>', '<s> imp <\\s>', '<s> no ...",1,0,0.0,"<re.Match object; span=(13, 33), match='no car..."


In [398]:
# create report list
cardio_report_0 = createReportList(cardio_df_0)
cardio_report_0[0]

['<s> cxr pa upright <\\s>',
 '<s> the chest shows round calcification at cardiac shadow should be calcification at left lower lung <\\s>',
 '<s> the right lung is clear <\\s>',
 '<s> the heart is not enlarged <\\s>',
 '<s> both costophrenic are clear <\\s>',
 '<s> the bony thorax is intact <\\s>']

Find N-grams Pattern

In [399]:
cardio_bigram_0 = createPatternNgrams(cardio_report_0, 2, cardio_keyword)
list(cardio_bigram_0.items())[:20]

[(('no', 'cardiomegaly'), 30027),
 (('<s>', 'no'), 29585),
 (('cardiomegaly', 'is'), 27373),
 (('is', 'observed'), 10752),
 (('observed', '<\\s>'), 10749),
 (('seen', '<\\s>'), 9948),
 (('is', 'seen'), 9921),
 (('noted', '<\\s>'), 3422),
 (('is', 'noted'), 3373),
 (('found', '<\\s>'), 3091),
 (('is', 'found'), 3089),
 (('cardiomegaly', '<\\s>'), 2211),
 (('cardiothoracic', 'ratio'), 658),
 (('<s>', 'there'), 546),
 (('there', 'is'), 531),
 (('heart', 'is'), 524),
 (('ratio', 'is'), 524),
 (('is', 'no'), 513),
 (('<s>', 'the'), 507),
 (('0.5', '<\\s>'), 470)]

In [400]:
cardio_trigram_0 = createPatternNgrams(cardio_report_0, 3, cardio_keyword)
list(cardio_trigram_0.items())[:20]

[(('<s>', 'no', 'cardiomegaly'), 29516),
 (('no', 'cardiomegaly', 'is'), 27370),
 (('is', 'observed', '<\\s>'), 10740),
 (('cardiomegaly', 'is', 'observed'), 10724),
 (('is', 'seen', '<\\s>'), 9893),
 (('cardiomegaly', 'is', 'seen'), 9771),
 (('is', 'noted', '<\\s>'), 3354),
 (('cardiomegaly', 'is', 'noted'), 3253),
 (('is', 'found', '<\\s>'), 3089),
 (('cardiomegaly', 'is', 'found'), 3086),
 (('no', 'cardiomegaly', '<\\s>'), 2206),
 (('<s>', 'there', 'is'), 526),
 (('cardiothoracic', 'ratio', 'is'), 519),
 (('there', 'is', 'no'), 497),
 (('is', 'no', 'cardiomegaly'), 494),
 (('ratio', 'is', 'about'), 465),
 (('about', '0.5', '<\\s>'), 451),
 (('is', 'about', '0.5'), 446),
 (('heart', 'is', 'not'), 401),
 (('<s>', 'cardiothoracic', 'ratio'), 336)]

## Ground truth 1

Prepare Data

In [401]:
# choose reports
cardio_df_1 = selectData(cardio_data, 'Cardiomegaly', 1.0)
cardio_df_1

,Reports,Report List,Cardiomegaly BERT Labeler,Cardiomegaly Inspectra Labeler,Cardiomegaly Ground Truth,Cardiomegaly Key Detect
0,CHEST :\nP.A. upright view .\nNo active pulmon...,"['<s> chest <\\s>', '<s> p.a <\\s>', '<s> upri...",1,0,1.0,"<re.Match object; span=(4, 24), match='lvh is ..."
5,CHEST\n\nNo definite pulmonary infiltration.\n...,"['<s> chest <\\s>', '<s> no definite pulmonary...",0,1,1.0,0.51
8,Chest;\n\n Left ventricular enlargement with ...,"['<s> chest <\\s>', '<s> left ventricular enla...",1,0,1.0,"<re.Match object; span=(4, 28), match='left ve..."
9,CHEST\n\nFibrotic change at left basal lung.\n...,"['<s> chest <\\s>', '<s> fibrotic change at le...",0,1,1.0,0.54
11,CHEST\n\nNo definite pulmonary infiltration.\n...,"['<s> chest <\\s>', '<s> no definite pulmonary...",0,1,1.0,0.54
...,...,...,...,...,...,...
39218,CHEST FILM\n\nNo pulmonary infiltration is see...,"['<s> chest film <\\s>', '<s> no pulmonary inf...",1,0,1.0,"<re.Match object; span=(4, 24), match='cardiom..."
39224,CHEST FILM\n\nNo pulmonary infiltration is see...,"['<s> chest film <\\s>', '<s> no pulmonary inf...",1,0,1.0,"<re.Match object; span=(4, 24), match='cardiom..."
39231,Chest PA upright\n \n No pulmonary infiltrat...,"['<s> chest pa upright <\\s>', '<s> no pulmona...",1,0,1.0,"<re.Match object; span=(4, 31), match='promine..."
39232,CXR PA \n\nNo definite pulmonary infiltration ...,"['<s> cxr pa <\\s>', '<s> no definite pulmonar...",0,1,1.0,0.51


In [402]:
# create report list
cardio_report_1 = createReportList(cardio_df_1)
cardio_report_1[0]

['<s> chest <\\s>',
 '<s> p.a <\\s>',
 '<s> upright view <\\s>',
 '<s> no active pulmonary disease is detected <\\s>',
 '<s> lvh is observed <\\s>',
 '<s> calcified and tortuous aortic arch are noted <\\s>']

Find N-grams Pattern

In [403]:
cardio_bigram_1 = createPatternNgrams(cardio_report_1, 2, cardio_keyword)
list(cardio_bigram_1.items())[:20]

[(('<s>', 'cardiomegaly'), 1383),
 (('noted', '<\\s>'), 1226),
 (('is', 'noted'), 1223),
 (('cardiomegaly', 'is'), 1203),
 (('cardiothoracic', 'ratio'), 974),
 (('<s>', 'prominent'), 887),
 (('ratio', 'is'), 445),
 (('ratio', '<\\s>'), 436),
 (('is', 'about'), 326),
 (('prominent', 'cardiac'), 319),
 (('prominent', 'cardiothoracic'), 306),
 (('size', '<\\s>'), 293),
 (('cardiac', 'shadow'), 290),
 (('prominent', 'heart'), 269),
 (('heart', 'size'), 246),
 (('shadow', '<\\s>'), 222),
 (('<s>', 'cardiothoracic'), 209),
 (('increased', 'cardiothoracic'), 201),
 (('<s>', 'increased'), 182),
 (('<s>', 'the'), 181)]

In [404]:
cardio_trigram_1 = createPatternNgrams(cardio_report_1, 3, cardio_keyword)
list(cardio_trigram_1.items())[:20]

[(('is', 'noted', '<\\s>'), 1213),
 (('<s>', 'cardiomegaly', 'is'), 1193),
 (('cardiomegaly', 'is', 'noted'), 1133),
 (('cardiothoracic', 'ratio', '<\\s>'), 435),
 (('cardiothoracic', 'ratio', 'is'), 417),
 (('ratio', 'is', 'about'), 326),
 (('<s>', 'prominent', 'cardiac'), 318),
 (('prominent', 'cardiothoracic', 'ratio'), 306),
 (('<s>', 'prominent', 'cardiothoracic'), 301),
 (('<s>', 'prominent', 'heart'), 237),
 (('prominent', 'heart', 'size'), 232),
 (('<s>', 'cardiothoracic', 'ratio'), 207),
 (('increased', 'cardiothoracic', 'ratio'), 200),
 (('cardiac', 'shadow', '<\\s>'), 193),
 (('prominent', 'cardiac', 'shadow'), 172),
 (('<s>', 'increased', 'cardiothoracic'), 171),
 (('heart', 'size', '<\\s>'), 160),
 (('<s>', 'the', 'cardiothoracic'), 155),
 (('the', 'cardiothoracic', 'ratio'), 155),
 (('about', '0.51', '<\\s>'), 149)]

## Ground truth -1

Prepare Data

In [405]:
# choose reports
cardio_df_u = selectData(cardio_data, 'Cardiomegaly', -1.0)
cardio_df_u

,Reports,Report List,Cardiomegaly BERT Labeler,Cardiomegaly Inspectra Labeler,Cardiomegaly Ground Truth,Cardiomegaly Key Detect
2,CXR PA upright\nNo demonstrable active pulmona...,"['<s> cxr pa upright <\\s>', '<s> no demonstra...",0,1,-1.0,"<re.Match object; span=(4, 32), match='borderl..."
3,CXR \n\nNo clinical history is provided.\n\nNo...,"['<s> cxr <\\s>', '<s> no clinical history is ...",0,1,-1.0,"<re.Match object; span=(13, 43), match='no cha..."
6,Chest PA upright;\n Rotate chest position.\n M...,"['<s> chest pa upright <\\s>', '<s> rotate che...",0,1,-1.0,"<re.Match object; span=(37, 57), match='mild p..."
17,CXR (PA upright)\n\nNo definite pulmonary inf...,"['<s> cxr pa upright <\\s>', '<s> no definite ...",1,0,-1.0,"<re.Match object; span=(4, 25), match='mild ca..."
21,Chest film\n\nReticulonodular infiltration bot...,"['<s> chest film <\\s>', '<s> reticulonodular ...",0,1,-1.0,"<re.Match object; span=(4, 32), match='borderl..."
...,...,...,...,...,...,...
39190,Chest:PA\n\nNo active pulmonary disease.\nDila...,"['<s> chest pa <\\s>', '<s> no active pulmonar...",0,1,-1.0,"<re.Match object; span=(4, 44), match='mild in..."
39200,"CXR\n\n Compare to CXR on 12/3/2018, markedly...","['<s> cxr <\\s>', '<s> compare to cxr on 12 3 ...",0,1,-1.0,"<re.Match object; span=(4, 64), match='borderl..."
39208,CXR PA upright\n\nHistory : --\n\nComparison s...,"['<s> cxr pa upright <\\s>', '<s> history <\\s...",1,0,-1.0,"<re.Match object; span=(4, 76), match='no chan..."
39209,"CXR PA upright\n\nHistory : DM,HT \n\nCompari...","['<s> cxr pa upright <\\s>', '<s> history dm h...",1,0,-1.0,"<re.Match object; span=(4, 40), match='no chan..."


In [406]:
# create report list
cardio_report_u = createReportList(cardio_df_u)
cardio_report_u[0]

['<s> cxr pa upright <\\s>',
 '<s> no demonstrable active pulmonary infiltration <\\s>',
 '<s> borderline cardiac size <\\s>',
 '<s> bony thorax is intact <\\s>',
 '<s> both costophrenic sulci are not remarkable <\\s>',
 '<s> imp no active pulmonary disease <\\s>']

Find N-grams Pattern

In [407]:
cardio_bigram_u = createPatternNgrams(cardio_report_u, 2, cardio_keyword)
list(cardio_bigram_u.items())[:20]

[(('<s>', 'mild'), 1410),
 (('mild', 'cardiomegaly'), 580),
 (('<s>', 'borderline'), 506),
 (('mild', 'prominent'), 504),
 (('prominent', 'heart'), 433),
 (('<s>', 'no'), 433),
 (('cardiomegaly', '<\\s>'), 359),
 (('size', '<\\s>'), 345),
 (('heart', 'size'), 322),
 (('cardiothoracic', 'ratio'), 319),
 (('aorta', '<\\s>'), 316),
 (('changed', 'of'), 307),
 (('no', 'changed'), 301),
 (('cardiac', 'size'), 255),
 (('vasculature', '<\\s>'), 240),
 (('of', 'cardiomegaly'), 236),
 (('pulmonary', 'vasculature'), 235),
 (('change', 'of'), 225),
 (('normal', 'pulmonary'), 224),
 (('borderline', 'cardiac'), 211)]

In [408]:
cardio_trigram_u = createPatternNgrams(cardio_report_u, 3, cardio_keyword)
list(cardio_trigram_u.items())[:20]

[(('<s>', 'mild', 'cardiomegaly'), 507),
 (('mild', 'prominent', 'heart'), 400),
 (('<s>', 'mild', 'prominent'), 400),
 (('prominent', 'heart', 'size'), 303),
 (('no', 'changed', 'of'), 297),
 (('<s>', 'no', 'changed'), 289),
 (('pulmonary', 'vasculature', '<\\s>'), 232),
 (('normal', 'pulmonary', 'vasculature'), 222),
 (('heart', 'size', '<\\s>'), 212),
 (('<s>', 'borderline', 'cardiac'), 211),
 (('borderline', 'cardiac', 'size'), 209),
 (('increased', 'cardiothoracic', 'ratio'), 201),
 (('mild', 'cardiomegaly', '<\\s>'), 195),
 (('no', 'change', 'of'), 176),
 (('mild', 'cardiomegaly', 'normal'), 165),
 (('cardiothoracic', 'ratio', '<\\s>'), 161),
 (('cardiomegaly', 'normal', 'pulmonary'), 158),
 (('<s>', 'mild', 'lvh'), 140),
 (('cardiac', 'size', 'with'), 138),
 (('changed', 'of', 'mild'), 133)]

## Unlabel

Prepare Data

In [409]:
# choose reports
cardio_df = selectData(cardio_data, 'Cardiomegaly', None)
cardio_df

,Reports,Report List,Cardiomegaly BERT Labeler,Cardiomegaly Inspectra Labeler,Cardiomegaly Ground Truth,Cardiomegaly Key Detect
4,Chest PA upright;\n R/O lung metastasis.\n\n E...,"['<s> chest pa upright <\\s>', '<s> r o lung m...",1,0,NaN,NaN
7,Chest PA uprihgt;\n Rt.lung destruction with f...,"['<s> chest pa uprihgt <\\s>', '<s> rt. lung d...",0,1,NaN,NaN
12,CXR PA upright\n The chest shows no definite ...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,NaN,NaN
15,CHEST PA UPRIGHT\n\nHistory: not available\nco...,"['<s> chest pa upright <\\s>', '<s> history no...",0,1,NaN,NaN
22,CHEST PA UPRIGHT\n\nHistory: not available\n\n...,"['<s> chest pa upright <\\s>', '<s> history no...",1,0,NaN,NaN
...,...,...,...,...,...,...
39184,CXR PA\n\n No active pulmonary infiltration i...,"['<s> cxr pa <\\s>', '<s> no active pulmonary ...",0,1,NaN,NaN
39185,CXR PA \n\nFINDINGS:\n - No demonstrabrle act...,"['<s> cxr pa <\\s>', '<s> findings <\\s>', '<s...",0,1,NaN,NaN
39193,CXR(PA)\n \n No active pulmonary infiltrati...,"['<s> cxr pa <\\s>', '<s> no active pulmonary ...",0,1,NaN,NaN
39206,CXR(PA upright)\n\n No pulmonary infiltration...,"['<s> cxr pa upright <\\s>', '<s> no pulmonary...",1,0,NaN,NaN


In [410]:
# create report list
cardio_report = createReportList(cardio_df)
cardio_report[0]

['<s> chest pa upright <\\s>',
 '<s> r o lung metastasis <\\s>',
 '<s> enlarged size and increased number of diffuse pulmonary nodules in both lungs are seen <\\s>',
 '<s> much more rt.pleural effusion is also shown <\\s>',
 '<s> ds <\\s>']

Find N-grams Pattern

In [411]:
cardio_bigram = createPatternNgrams(cardio_report, 2)
list(cardio_bigram.items())[:20]

[(('<s>', 'no'), 1856),
 (('pleural', 'effusion'), 953),
 (('noted', '<\\s>'), 801),
 (('<s>', 'the'), 798),
 (('seen', '<\\s>'), 756),
 (('is', 'noted'), 737),
 (('is', 'seen'), 722),
 (('pulmonary', 'infiltration'), 716),
 (('no', 'pleural'), 664),
 (('bony', 'thorax'), 623),
 (('intact', '<\\s>'), 536),
 (('thorax', 'is'), 475),
 (('effusion', 'is'), 469),
 (('infiltration', 'or'), 433),
 (('is', 'intact'), 430),
 (('clear', '<\\s>'), 427),
 (('<s>', 'bony'), 402),
 (('no', 'active'), 391),
 (('active', 'pulmonary'), 390),
 (('no', 'pulmonary'), 374)]

In [412]:
cardio_trigram = createPatternNgrams(cardio_report, 3)
list(cardio_trigram.items())[:20]

[(('is', 'seen', '<\\s>'), 679),
 (('is', 'noted', '<\\s>'), 679),
 (('no', 'pleural', 'effusion'), 663),
 (('<s>', 'no', 'pleural'), 639),
 (('bony', 'thorax', 'is'), 472),
 (('is', 'intact', '<\\s>'), 429),
 (('pulmonary', 'infiltration', 'or'), 423),
 (('pleural', 'effusion', 'is'), 422),
 (('thorax', 'is', 'intact'), 388),
 (('effusion', 'is', 'seen'), 373),
 (('<s>', 'no', 'pulmonary'), 363),
 (('infiltration', 'or', 'nodule'), 354),
 (('no', 'pulmonary', 'infiltration'), 339),
 (('are', 'clear', '<\\s>'), 331),
 (('<s>', 'the', 'heart'), 323),
 (('no', 'active', 'pulmonary'), 322),
 (('or', 'nodule', 'is'), 322),
 (('<s>', 'bony', 'thorax'), 319),
 (('nodule', 'is', 'noted'), 313),
 (('pleural', 'effusion', '<\\s>'), 285)]

In [413]:
cardio_fourgram = createPatternNgrams(cardio_report, 4)
list(cardio_fourgram.items())[:20]

[(('<s>', 'no', 'pleural', 'effusion'), 638),
 (('thorax', 'is', 'intact', '<\\s>'), 387),
 (('bony', 'thorax', 'is', 'intact'), 385),
 (('effusion', 'is', 'seen', '<\\s>'), 373),
 (('no', 'pleural', 'effusion', 'is'), 364),
 (('pulmonary', 'infiltration', 'or', 'nodule'), 349),
 (('pleural', 'effusion', 'is', 'seen'), 348),
 (('<s>', 'no', 'pulmonary', 'infiltration'), 330),
 (('infiltration', 'or', 'nodule', 'is'), 320),
 (('no', 'pulmonary', 'infiltration', 'or'), 314),
 (('nodule', 'is', 'noted', '<\\s>'), 309),
 (('or', 'nodule', 'is', 'noted'), 306),
 (('<s>', 'bony', 'thorax', 'is'), 276),
 (('<s>', 'the', 'heart', 'size'), 257),
 (('the', 'heart', 'size', 'appears'), 253),
 (('heart', 'size', 'appears', 'normal'), 253),
 (('angles', 'are', 'clear', '<\\s>'), 252),
 (('size', 'appears', 'normal', '<\\s>'), 252),
 (('costophrenic', 'angles', 'are', 'clear'), 236),
 (('both', 'costophrenic', 'angles', 'are'), 220)]

In [414]:
cardio_bigramk = createPatternNgrams(cardio_report, 2, cardio_keyword)
list(cardio_bigramk.items())[:20]

[(('<s>', 'the'), 356),
 (('the', 'heart'), 327),
 (('heart', 'size'), 303),
 (('appears', 'normal'), 278),
 (('<s>', 'no'), 267),
 (('normal', '<\\s>'), 259),
 (('size', 'appears'), 253),
 (('cardiomegaly', '<\\s>'), 215),
 (('or', 'cardiomegaly'), 202),
 (('pleural', 'effusion'), 193),
 (('effusion', 'or'), 189),
 (('no', 'pleural'), 179),
 (('<s>', 'heart'), 143),
 (('size', '<\\s>'), 92),
 (('cardiomegaly', 'is'), 73),
 (('heart', 'and'), 66),
 (('normal', 'size'), 58),
 (('seen', '<\\s>'), 57),
 (('is', 'noted'), 56),
 (('is', 'seen'), 55)]

In [415]:
cardio_trigramk = createPatternNgrams(cardio_report, 3, cardio_keyword)
list(cardio_trigramk.items())[:20]

[(('<s>', 'the', 'heart'), 323),
 (('the', 'heart', 'size'), 257),
 (('heart', 'size', 'appears'), 253),
 (('size', 'appears', 'normal'), 253),
 (('appears', 'normal', '<\\s>'), 253),
 (('effusion', 'or', 'cardiomegaly'), 187),
 (('no', 'pleural', 'effusion'), 179),
 (('pleural', 'effusion', 'or'), 179),
 (('or', 'cardiomegaly', '<\\s>'), 174),
 (('<s>', 'no', 'pleural'), 171),
 (('normal', 'size', '<\\s>'), 50),
 (('is', 'seen', '<\\s>'), 49),
 (('is', 'noted', '<\\s>'), 43),
 (('<s>', 'there', 'is'), 41),
 (('<s>', 'heart', 'and'), 36),
 (('heart', 'and', 'mediastinum'), 35),
 (('cardiomegaly', 'is', 'seen'), 33),
 (('<s>', 'no', 'significant'), 30),
 (('no', 'significant', 'cardiomegaly'), 30),
 (('no', 'definite', 'cardiomegaly'), 29)]

In [416]:
cardio_fourgramk = createPatternNgrams(cardio_report, 4, cardio_keyword)
list(cardio_fourgramk.items())[:20]

[(('<s>', 'the', 'heart', 'size'), 257),
 (('the', 'heart', 'size', 'appears'), 253),
 (('heart', 'size', 'appears', 'normal'), 253),
 (('size', 'appears', 'normal', '<\\s>'), 252),
 (('pleural', 'effusion', 'or', 'cardiomegaly'), 178),
 (('no', 'pleural', 'effusion', 'or'), 176),
 (('<s>', 'no', 'pleural', 'effusion'), 171),
 (('effusion', 'or', 'cardiomegaly', '<\\s>'), 160),
 (('cardiomegaly', 'is', 'seen', '<\\s>'), 33),
 (('<s>', 'no', 'significant', 'cardiomegaly'), 30),
 (('effusion', 'or', 'cardiomegaly', 'is'), 27),
 (('cardiomegaly', 'is', 'noted', '<\\s>'), 25),
 (('<s>', 'the', 'heart', 'and'), 23),
 (('heart', 'and', 'great', 'vessels'), 21),
 (('appears', 'normal', 'size', '<\\s>'), 21),
 (('<s>', 'heart', 'and', 'mediastinum'), 21),
 (('heart', 'has', 'normal', 'size'), 21),
 (('<s>', 'no', 'definite', 'cardiomegaly'), 20),
 (('or', 'cardiomegaly', 'is', 'seen'), 20),
 (('has', 'normal', 'size', '<\\s>'), 20)]

In [417]:
cardio_fivegramk = createPatternNgrams(cardio_report, 5, cardio_keyword)
list(cardio_fivegramk.items())[:20]

[(('<s>', 'the', 'heart', 'size', 'appears'), 253),
 (('the', 'heart', 'size', 'appears', 'normal'), 253),
 (('heart', 'size', 'appears', 'normal', '<\\s>'), 252),
 (('no', 'pleural', 'effusion', 'or', 'cardiomegaly'), 175),
 (('<s>', 'no', 'pleural', 'effusion', 'or'), 169),
 (('pleural', 'effusion', 'or', 'cardiomegaly', '<\\s>'), 154),
 (('pleural', 'effusion', 'or', 'cardiomegaly', 'is'), 24),
 (('effusion', 'or', 'cardiomegaly', 'is', 'seen'), 20),
 (('or', 'cardiomegaly', 'is', 'seen', '<\\s>'), 20),
 (('<s>', 'no', 'significant', 'cardiomegaly', '<\\s>'), 19),
 (('<s>', 'no', 'definite', 'cardiomegaly', 'is'), 18),
 (('heart', 'has', 'normal', 'size', '<\\s>'), 16),
 (('cardiothoracic', 'ratio', 'is', 'in', 'normal'), 14),
 (('ratio', 'is', 'in', 'normal', 'range'), 14),
 (('is', 'in', 'normal', 'range', '<\\s>'), 14),
 (('<s>', 'heart', 'size', 'cannot', 'be'), 14),
 (('<s>', 'the', 'heart', 'has', 'normal'), 13),
 (('the', 'heart', 'has', 'normal', 'size'), 13),
 (('heart', 'a

In [418]:
cardio_sixgramk = createPatternNgrams(cardio_report, 6, cardio_keyword)
list(cardio_sixgramk.items())[:20]

[(('<s>', 'the', 'heart', 'size', 'appears', 'normal'), 253),
 (('the', 'heart', 'size', 'appears', 'normal', '<\\s>'), 252),
 (('<s>', 'no', 'pleural', 'effusion', 'or', 'cardiomegaly'), 168),
 (('no', 'pleural', 'effusion', 'or', 'cardiomegaly', '<\\s>'), 151),
 (('no', 'pleural', 'effusion', 'or', 'cardiomegaly', 'is'), 24),
 (('effusion', 'or', 'cardiomegaly', 'is', 'seen', '<\\s>'), 20),
 (('pleural', 'effusion', 'or', 'cardiomegaly', 'is', 'seen'), 17),
 (('cardiothoracic', 'ratio', 'is', 'in', 'normal', 'range'), 14),
 (('ratio', 'is', 'in', 'normal', 'range', '<\\s>'), 14),
 (('<s>', 'the', 'heart', 'has', 'normal', 'size'), 13),
 (('<s>', 'cardiothoracic', 'ratio', 'is', 'in', 'normal'), 12),
 (('<s>', 'the', 'heart', 'and', 'great', 'vessels'), 10),
 (('<s>', 'heart', 'appears', 'normal', 'size', '<\\s>'), 10),
 (('<s>', 'no', 'plerual', 'effusion', 'or', 'cardiomegaly'), 9),
 (('<s>', 'heart', 'and', 'mediastinum', 'shift', 'to'), 9),
 (('heart', 'and', 'mediastinum', 'shift

# Pleural Effusion

In [419]:
# define keyword
pleural_keyword = r'pleural effusion|costophrenic'

## Ground truth 0

Prepare Data

In [420]:
# choose reports
pleural_df_0 = selectData(pleural_data, 'Pleural Effusion', 0.0)
pleural_df_0

,Reports,Report List,Pleural Effusion BERT Labeler,Pleural Effusion Inspectra Labeler,Pleural Effusion Ground Truth,Pleural Effusion Key Detect
7,CXR\nInterstitial infitration both lungs\nMild...,"['<s> cxr <\\s>', '<s> interstitial infitratio...",1,0,0.0,"<re.Match object; span=(0, 36), match='<s> bot..."
8,Chest PA upright;\n compared to film on 20/4/2...,"['<s> chest pa upright <\\s>', '<s> compared t...",1,0,0.0,"<re.Match object; span=(0, 33), match='<s> rt...."
11,CXR PA upright (13/7/2005)\n\n Fibros...,"['<s> cxr pa upright 13 7 2005 <\\s>', '<s> fi...",1,0,0.0,"<re.Match object; span=(0, 36), match='<s> bot..."
18,Chest film\n\nMinimal fibrosis in RUL and inte...,"['<s> chest film <\\s>', '<s> minimal fibrosis...",1,0,0.0,"<re.Match object; span=(0, 36), match='<s> bot..."
21,CXR PA upright\n By compared with previous ch...,"['<s> cxr pa upright <\\s>', '<s> by compared ...",1,0,0.0,"<re.Match object; span=(0, 43), match='<s> bot..."
...,...,...,...,...,...,...
4544,CXR PA \n\nMinimal fibrosis at LLL.\nNormal ca...,"['<s> cxr pa <\\s>', '<s> minimal fibrosis at ...",1,0,0.0,"<re.Match object; span=(0, 36), match='<s> bot..."
4547,CXR\n\nAs compared to prior film on 12 march 1...,"['<s> cxr <\\s>', '<s> as compared to prior fi...",0,1,0.0,"<re.Match object; span=(58, 102), match='no de..."
4551,CXR PA upright\n\n Compared to prior film on 3...,"['<s> cxr pa upright <\\s>', '<s> compared to ...",1,0,0.0,"<re.Match object; span=(4, 45), match='no defi..."
4558,CXR PA upright\n\nComparison to prior radiogra...,"['<s> cxr pa upright <\\s>', '<s> comparison t...",0,1,0.0,"<re.Match object; span=(4, 147), match='no int..."


In [421]:
# create report list
pleural_report_0 = createReportList(pleural_df_0)
pleural_report_0[0]

['<s> cxr <\\s>',
 '<s> interstitial infitration both lungs <\\s>',
 '<s> mild cardiomegaly <\\s>',
 '<s> both costophrenic sulci are not remarkable <\\s>']

Find N-grams Pattern

In [422]:
pleural_bigram_0 = createPatternNgrams(pleural_report_0, 2, pleural_keyword)
list(pleural_bigram_0.items())[:20]

[(('pleural', 'effusion'), 485),
 (('both', 'costophrenic'), 396),
 (('<s>', 'both'), 392),
 (('clear', '<\\s>'), 355),
 (('are', 'clear'), 316),
 (('effusion', '<\\s>'), 274),
 (('<s>', 'no'), 256),
 (('costophrenic', 'angles'), 230),
 (('angles', 'are'), 230),
 (('change', 'of'), 217),
 (('costophrenic', 'sulci'), 211),
 (('sulci', 'are'), 177),
 (('right', 'pleural'), 167),
 (('not', 'remarkable'), 166),
 (('remarkable', '<\\s>'), 166),
 (('are', 'not'), 165),
 (('no', 'interval'), 138),
 (('interval', 'change'), 129),
 (('left', 'pleural'), 123),
 (('effusion', 'is'), 122)]

In [423]:
pleural_trigram_0 = createPatternNgrams(pleural_report_0, 3, pleural_keyword)
list(pleural_trigram_0.items())[:20]

[(('<s>', 'both', 'costophrenic'), 392),
 (('are', 'clear', '<\\s>'), 316),
 (('pleural', 'effusion', '<\\s>'), 264),
 (('costophrenic', 'angles', 'are'), 230),
 (('angles', 'are', 'clear'), 227),
 (('both', 'costophrenic', 'angles'), 222),
 (('costophrenic', 'sulci', 'are'), 177),
 (('both', 'costophrenic', 'sulci'), 170),
 (('right', 'pleural', 'effusion'), 167),
 (('not', 'remarkable', '<\\s>'), 166),
 (('are', 'not', 'remarkable'), 165),
 (('sulci', 'are', 'not'), 129),
 (('no', 'interval', 'change'), 129),
 (('interval', 'change', 'of'), 129),
 (('left', 'pleural', 'effusion'), 120),
 (('<s>', 'no', 'interval'), 120),
 (('of', 'right', 'pleural'), 114),
 (('pleural', 'effusion', 'is'), 106),
 (('change', 'of', 'right'), 93),
 (('<s>', 'neither', 'pleural'), 72)]

## Ground truth 1

Prepare Data

In [424]:
# choose reports
pleural_df_1 = selectData(pleural_data, 'Pleural Effusion', 1.0)
pleural_df_1

,Reports,Report List,Pleural Effusion BERT Labeler,Pleural Effusion Inspectra Labeler,Pleural Effusion Ground Truth,Pleural Effusion Key Detect
4,CXR PA upright\n\nPulmonary infiltration at RL...,"['<s> cxr pa upright <\\s>', '<s> pulmonary in...",0,1,1.0,"<re.Match object; span=(38, 60), match='right ..."
9,CHEST PA UPRIGHT\n\nRight pleural effusion and...,"['<s> chest pa upright <\\s>', '<s> right pleu...",1,0,1.0,"<re.Match object; span=(4, 26), match='right p..."
10,CHEST PA UPRIGHT\n\nThere is righ tpleural eff...,"['<s> chest pa upright <\\s>', '<s> there is r...",1,0,1.0,"<re.Match object; span=(13, 35), match='righ t..."
17,CXR (PA_upright)\n\nFibrosis with reticulonod...,"['<s> cxr pa_upright <\\s>', '<s> fibrosis wit...",0,1,1.0,"<re.Match object; span=(4, 26), match='right p..."
25,CXR(PA)\n \nClinical history: loculated ple...,"['<s> cxr pa <\\s>', '<s> clinical history loc...",1,0,1.0,"<re.Match object; span=(0, 47), match='<s> cli..."
...,...,...,...,...,...,...
4497,Chest PA upright \n\nHistory: Rt.pleural effus...,"['<s> chest pa upright <\\s>', '<s> history rt...",1,0,1.0,"<re.Match object; span=(0, 31), match='<s> his..."
4549,CXR PA upright\n\nLarge bleb at RUL\nOpacity a...,"['<s> cxr pa upright <\\s>', '<s> large bleb a...",0,1,1.0,"<re.Match object; span=(4, 26), match='right p..."
4555,Chest PA upright \n\nPrior noted bilateral ple...,"['<s> chest pa upright <\\s>', '<s> prior note...",1,0,1.0,"<re.Match object; span=(10, 42), match='noted ..."
4556,CHEST FILM\n\nReticulonodular infiltration at ...,"['<s> chest film <\\s>', '<s> reticulonodular ...",1,0,1.0,"<re.Match object; span=(20, 41), match='left p..."


In [425]:
# create report list
pleural_report_1 = createReportList(pleural_df_1)
pleural_report_1[0]

['<s> cxr pa upright <\\s>',
 '<s> pulmonary infiltration at rll and right pleural effusion <\\s>',
 '<s> normal cardiothoracic ratio <\\s>',
 '<s> bony thorax is intact <\\s>']

Find N-grams Pattern

In [426]:
pleural_bigram_1 = createPatternNgrams(pleural_report_1, 2, pleural_keyword)
list(pleural_bigram_1.items())[:20]

[(('pleural', 'effusion'), 349),
 (('effusion', '<\\s>'), 259),
 (('right', 'pleural'), 140),
 (('left', 'pleural'), 101),
 (('<s>', 'right'), 71),
 (('effusion', 'is'), 59),
 (('bilateral', 'pleural'), 59),
 (('<s>', 'left'), 57),
 (('<s>', 'bilateral'), 38),
 (('<s>', 'small'), 30),
 (('and', 'right'), 29),
 (('is', 'seen'), 29),
 (('costophrenic', 'angle'), 29),
 (('infiltration', 'at'), 28),
 (('seen', '<\\s>'), 28),
 (('<s>', 'no'), 26),
 (('<s>', 'pulmonary'), 24),
 (('pulmonary', 'infiltration'), 24),
 (('is', 'noted'), 24),
 (('noted', '<\\s>'), 24)]

In [427]:
pleural_trigram_1 = createPatternNgrams(pleural_report_1, 3, pleural_keyword)
list(pleural_trigram_1.items())[:20]

[(('pleural', 'effusion', '<\\s>'), 257),
 (('right', 'pleural', 'effusion'), 137),
 (('left', 'pleural', 'effusion'), 101),
 (('<s>', 'right', 'pleural'), 66),
 (('pleural', 'effusion', 'is'), 58),
 (('<s>', 'left', 'pleural'), 50),
 (('bilateral', 'pleural', 'effusion'), 50),
 (('<s>', 'bilateral', 'pleural'), 35),
 (('is', 'seen', '<\\s>'), 27),
 (('effusion', 'is', 'seen'), 26),
 (('and', 'right', 'pleural'), 25),
 (('<s>', 'pulmonary', 'infiltration'), 24),
 (('pulmonary', 'infiltration', 'at'), 23),
 (('is', 'noted', '<\\s>'), 23),
 (('infiltration', 'at', 'rll'), 21),
 (('costophrenic', 'angle', 'is'), 21),
 (('at', 'rll', 'and'), 19),
 (('rll', 'and', 'right'), 19),
 (('effusion', 'is', 'noted'), 17),
 (('left', 'costophrenic', 'angle'), 16)]

## Ground truth -1

Prepare Data

In [428]:
# choose reports
pleural_df_u = selectData(pleural_data, 'Pleural Effusion', -1.0)
pleural_df_u

,Reports,Report List,Pleural Effusion BERT Labeler,Pleural Effusion Inspectra Labeler,Pleural Effusion Ground Truth,Pleural Effusion Key Detect
0,"Chest PA upright;\n follow up film , compared ...","['<s> chest pa upright <\\s>', '<s> follow up ...",0,1,-1.0,"<re.Match object; span=(4, 49), match='no sign..."
1,Chest;\n\n Minimal fibrosis at RUL .\n The l...,"['<s> chest <\\s>', '<s> minimal fibrosis at r...",1,0,-1.0,"<re.Match object; span=(0, 42), match='<s> bot..."
2,CHEST\n\nReticulonodular infiltration at RUL. ...,"['<s> chest <\\s>', '<s> reticulonodular infil...",0,1,-1.0,"<re.Match object; span=(4, 48), match='no sign..."
5,CXR\n\nHistory of CA lung\n\nAs compared to pr...,"['<s> cxr <\\s>', '<s> history of ca lung <\\s...",0,1,-1.0,"<re.Match object; span=(56, 91), match='no cha..."
12,CXR(PA)\n \n History: Pleural TB. Foll...,"['<s> cxr pa <\\s>', '<s> history pleural tb. ...",0,1,-1.0,"<re.Match object; span=(49, 100), match='no si..."
...,...,...,...,...,...,...
4548,"CXR\n\n Compare to CXR on 30/04/2017, no si...","['<s> cxr <\\s>', '<s> compare to cxr on 30 04...",0,1,-1.0,"<re.Match object; span=(33, 80), match='no sig..."
4550,CXR PA\n\nCompared with previous study: Jan 2...,"['<s> cxr pa <\\s>', '<s> compared with previo...",0,1,-1.0,"<re.Match object; span=(4, 55), match='no sign..."
4552,CXR PA upright\n\n Compared with previous che...,"['<s> cxr pa upright <\\s>', '<s> compared wit...",0,1,-1.0,"<re.Match object; span=(20, 63), match='no cha..."
4553,CXR PA upright\n\n Compared with previous che...,"['<s> cxr pa upright <\\s>', '<s> compared wit...",0,1,-1.0,"<re.Match object; span=(20, 60), match='no cha..."


In [429]:
# create report list
pleural_report_u = createReportList(pleural_df_u)
pleural_report_u[0]

['<s> chest pa upright <\\s>',
 '<s> follow up film compared to film on 22 3 2005 <\\s>',
 '<s> no significant changed of rt.pleural effusion opacity of rll <\\s>',
 '<s> l.t lung is clear <\\s>',
 '<s> remaining thickening of rt.paratracheal soft tissue is also shown <\\s>',
 '<s> heart is mild prominent in size <\\s>',
 '<s> tortuous thoracic aorta is seen <\\s>',
 '<s> ds <\\s>']

Find N-grams Pattern

In [430]:
pleural_bigram_u = createPatternNgrams(pleural_report_u, 2, pleural_keyword)
list(pleural_bigram_u.items())[:20]

[(('pleural', 'effusion'), 2271),
 (('change', 'of'), 1788),
 (('<s>', 'no'), 1590),
 (('effusion', '<\\s>'), 1341),
 (('right', 'pleural'), 1068),
 (('no', 'change'), 1029),
 (('no', 'significant'), 797),
 (('left', 'pleural'), 737),
 (('significant', 'change'), 731),
 (('of', 'right'), 603),
 (('costophrenic', 'angle'), 567),
 (('of', 'left'), 422),
 (('left', 'costophrenic'), 346),
 (('effusion', 'is'), 328),
 (('<s>', 'the'), 324),
 (('there', 'is'), 318),
 (('clear', '<\\s>'), 312),
 (('is', 'no'), 296),
 (('angle', 'is'), 285),
 (('right', 'costophrenic'), 262)]

In [431]:
pleural_trigram_u = createPatternNgrams(pleural_report_u, 3, pleural_keyword)
list(pleural_trigram_u.items())[:20]

[(('pleural', 'effusion', '<\\s>'), 1294),
 (('right', 'pleural', 'effusion'), 1050),
 (('no', 'change', 'of'), 1000),
 (('no', 'significant', 'change'), 731),
 (('significant', 'change', 'of'), 726),
 (('left', 'pleural', 'effusion'), 720),
 (('<s>', 'no', 'change'), 599),
 (('<s>', 'no', 'significant'), 588),
 (('of', 'right', 'pleural'), 500),
 (('change', 'of', 'right'), 424),
 (('of', 'left', 'pleural'), 320),
 (('pleural', 'effusion', 'is'), 317),
 (('left', 'costophrenic', 'angle'), 317),
 (('there', 'is', 'no'), 292),
 (('costophrenic', 'angle', 'is'), 276),
 (('change', 'of', 'left'), 272),
 (('<s>', 'there', 'is'), 247),
 (('pleural', 'effusion', 'and'), 238),
 (('right', 'costophrenic', 'angle'), 238),
 (('bilateral', 'pleural', 'effusion'), 222)]

## Unlabel

Prepare Data

In [432]:
# choose reports
pleural_df = selectData(pleural_data, 'Pleural Effusion', None)
pleural_df

,Reports,Report List,Pleural Effusion BERT Labeler,Pleural Effusion Inspectra Labeler,Pleural Effusion Ground Truth,Pleural Effusion Key Detect
3,CHEST\n\nRt hydropneumothorax is seen.\nLeft l...,"['<s> chest <\\s>', '<s> rt hydropneumothorax ...",0,1,NaN,NaN
6,CHEST\n\nThere is both upper lobes infiltratio...,"['<s> chest <\\s>', '<s> there is both upper l...",1,0,NaN,NaN
13,CHEST.;\nLoss of right breast shadow.\nMassive...,"['<s> chest <\\s>', '<s> loss of right breast ...",1,0,NaN,NaN
15,CHEST\n\nS/P sternotomy.\nCompared to previous...,"['<s> chest <\\s>', '<s> s p sternotomy <\\s>'...",0,1,NaN,NaN
19,Chest ( PA upright view)\n As compared with ...,"['<s> chest pa upright view <\\s>', '<s> as co...",0,1,NaN,NaN
...,...,...,...,...,...,...
4502,CXR PA upright\n\nHistory : --\n\nComparison s...,"['<s> cxr pa upright <\\s>', '<s> history <\\s...",0,1,NaN,NaN
4504,CXR PA\n\nCompared with previous study: 07/12/...,"['<s> cxr pa <\\s>', '<s> compared with previo...",1,0,NaN,NaN
4545,Chest PA upright\n\nComparison: 3/4/2018\n\nFi...,"['<s> chest pa upright <\\s>', '<s> comparison...",0,1,NaN,NaN
4554,Chest PA upright \n\ncompared to film on 25/1/...,"['<s> chest pa upright <\\s>', '<s> compared t...",0,1,NaN,NaN


In [433]:
# create report list
pleural_report = createReportList(pleural_df)
pleural_report[0]

['<s> chest <\\s>',
 '<s> rt hydropneumothorax is seen <\\s>',
 '<s> left lung is clear <\\s>',
 '<s> no cardiomegaly is noted <\\s>']

Find N-grams Pattern

In [434]:
pleural_bigram = createPatternNgrams(pleural_report, 2)
list(pleural_bigram.items())[:20]

[(('<s>', 'no'), 694),
 (('<s>', 'normal'), 472),
 (('<s>', 'unremarkable'), 374),
 (('<s>', 'bony'), 326),
 (('<s>', 'the'), 267),
 (('change', 'of'), 254),
 (('unremarkable', '<\\s>'), 254),
 (('<s>', 'findings'), 250),
 (('findings', '<\\s>'), 248),
 (('size', '<\\s>'), 245),
 (('<s>', 'heart'), 245),
 (('pulmonary', 'infiltration'), 244),
 (('<s>', 'there'), 240),
 (('heart', 'size'), 238),
 (('normal', 'heart'), 230),
 (('none', '<\\s>'), 230),
 (('bony', 'structures'), 223),
 (('there', 'is'), 214),
 (('plerual', 'effusion'), 211),
 (('infiltration', '<\\s>'), 209)]

In [435]:
pleural_trigram = createPatternNgrams(pleural_report, 3)
list(pleural_trigram.items())[:20]

[(('<s>', 'findings', '<\\s>'), 248),
 (('<s>', 'normal', 'heart'), 219),
 (('normal', 'heart', 'size'), 219),
 (('<s>', 'bony', 'structures'), 209),
 (('heart', 'size', '<\\s>'), 204),
 (('<s>', 'there', 'is'), 200),
 (('bony', 'structures', '<\\s>'), 200),
 (('<s>', 'impression', '<\\s>'), 199),
 (('<s>', 'heart', 'and'), 196),
 (('heart', 'and', 'mediastinum'), 195),
 (('<s>', 'none', '<\\s>'), 194),
 (('effusion', 'nor', 'pneumothorax'), 192),
 (('<s>', 'normal', 'mediastinal'), 192),
 (('<s>', 'lung', 'parenchyma'), 191),
 (('lung', 'parenchyma', '<\\s>'), 191),
 (('<s>', 'pleura', '<\\s>'), 191),
 (('<s>', 'neither', 'plerual'), 191),
 (('neither', 'plerual', 'effusion'), 191),
 (('plerual', 'effusion', 'nor'), 191),
 (('nor', 'pneumothorax', '<\\s>'), 191)]

In [436]:
pleural_fourgram = createPatternNgrams(pleural_report, 4)
list(pleural_fourgram.items())[:20]

[(('<s>', 'normal', 'heart', 'size'), 219),
 (('normal', 'heart', 'size', '<\\s>'), 201),
 (('<s>', 'heart', 'and', 'mediastinum'), 194),
 (('<s>', 'lung', 'parenchyma', '<\\s>'), 191),
 (('<s>', 'neither', 'plerual', 'effusion'), 191),
 (('neither', 'plerual', 'effusion', 'nor'), 191),
 (('plerual', 'effusion', 'nor', 'pneumothorax'), 191),
 (('effusion', 'nor', 'pneumothorax', '<\\s>'), 191),
 (('heart', 'and', 'mediastinum', '<\\s>'), 191),
 (('<s>', 'normal', 'mediastinal', 'contour'), 191),
 (('normal', 'mediastinal', 'contour', '<\\s>'), 191),
 (('<s>', 'bony', 'structures', '<\\s>'), 191),
 (('<s>', 'tube', 'and', 'line'), 190),
 (('tube', 'and', 'line', '<\\s>'), 190),
 (('<s>', 'unremarkable', 'for', 'age'), 179),
 (('unremarkable', 'for', 'age', '<\\s>'), 179),
 (('<s>', 'no', 'pulmonary', 'infiltration'), 175),
 (('no', 'pulmonary', 'infiltration', '<\\s>'), 163),
 (('<s>', 'no', 'detectable', 'abnormality'), 151),
 (('no', 'detectable', 'abnormality', '<\\s>'), 151)]

In [437]:
pleural_bigramk = createPatternNgrams(pleural_report, 2, pleural_keyword)
list(pleural_bigramk.items())[:20]

[(('both', 'costophrenic'), 10),
 (('<s>', 'both'), 9),
 (('costophrenic', 'sulci'), 9),
 (('sulci', 'appear'), 8),
 (('appear', 'unremarkable'), 8),
 (('unremarkable', '<\\s>'), 8),
 (('costophrenic', 'angle'), 6),
 (('pleural', 'effusion'), 5),
 (('angle', 'is'), 3),
 (('left', 'pleural'), 3),
 (('resolved', '<\\s>'), 3),
 (('effusion', 'are'), 3),
 (('<s>', 'the'), 2),
 (('<s>', 'loss'), 2),
 (('loss', 'of'), 2),
 (('of', 'costophrenic'), 2),
 (('angle', 'with'), 2),
 (('with', 'fluid'), 2),
 (('fluid', 'in'), 2),
 (('in', 'left'), 2)]

In [438]:
pleural_trigramk = createPatternNgrams(pleural_report, 3, pleural_keyword)
list(pleural_trigramk.items())[:20]

[(('<s>', 'both', 'costophrenic'), 9),
 (('both', 'costophrenic', 'sulci'), 9),
 (('costophrenic', 'sulci', 'appear'), 8),
 (('sulci', 'appear', 'unremarkable'), 8),
 (('appear', 'unremarkable', '<\\s>'), 8),
 (('costophrenic', 'angle', 'is'), 3),
 (('pleural', 'effusion', 'are'), 3),
 (('<s>', 'loss', 'of'), 2),
 (('loss', 'of', 'costophrenic'), 2),
 (('of', 'costophrenic', 'angle'), 2),
 (('costophrenic', 'angle', 'with'), 2),
 (('angle', 'with', 'fluid'), 2),
 (('with', 'fluid', 'in'), 2),
 (('fluid', 'in', 'left'), 2),
 (('in', 'left', 'pleural'), 2),
 (('left', 'pleural', 'space'), 2),
 (('pleural', 'space', '<\\s>'), 2),
 (('bilateral', 'pleural', 'effusion'), 2),
 (('<s>', 'the', 'left'), 1),
 (('the', 'left', 'costophrenic'), 1)]

In [439]:
pleural_fourgramk = createPatternNgrams(pleural_report, 4, pleural_keyword)
list(pleural_fourgramk.items())[:20]

[(('<s>', 'both', 'costophrenic', 'sulci'), 9),
 (('both', 'costophrenic', 'sulci', 'appear'), 8),
 (('costophrenic', 'sulci', 'appear', 'unremarkable'), 8),
 (('sulci', 'appear', 'unremarkable', '<\\s>'), 8),
 (('<s>', 'loss', 'of', 'costophrenic'), 2),
 (('loss', 'of', 'costophrenic', 'angle'), 2),
 (('of', 'costophrenic', 'angle', 'with'), 2),
 (('costophrenic', 'angle', 'with', 'fluid'), 2),
 (('angle', 'with', 'fluid', 'in'), 2),
 (('with', 'fluid', 'in', 'left'), 2),
 (('fluid', 'in', 'left', 'pleural'), 2),
 (('in', 'left', 'pleural', 'space'), 2),
 (('left', 'pleural', 'space', '<\\s>'), 2),
 (('bilateral', 'pleural', 'effusion', 'are'), 2),
 (('<s>', 'the', 'left', 'costophrenic'), 1),
 (('the', 'left', 'costophrenic', 'angle'), 1),
 (('left', 'costophrenic', 'angle', 'is'), 1),
 (('costophrenic', 'angle', 'is', 'still'), 1),
 (('angle', 'is', 'still', 'clear'), 1),
 (('is', 'still', 'clear', '<\\s>'), 1)]

In [440]:
pleural_fivegramk = createPatternNgrams(pleural_report, 5, pleural_keyword)
list(pleural_fivegramk.items())[:20]

[(('<s>', 'both', 'costophrenic', 'sulci', 'appear'), 8),
 (('both', 'costophrenic', 'sulci', 'appear', 'unremarkable'), 8),
 (('costophrenic', 'sulci', 'appear', 'unremarkable', '<\\s>'), 8),
 (('<s>', 'loss', 'of', 'costophrenic', 'angle'), 2),
 (('loss', 'of', 'costophrenic', 'angle', 'with'), 2),
 (('of', 'costophrenic', 'angle', 'with', 'fluid'), 2),
 (('costophrenic', 'angle', 'with', 'fluid', 'in'), 2),
 (('angle', 'with', 'fluid', 'in', 'left'), 2),
 (('with', 'fluid', 'in', 'left', 'pleural'), 2),
 (('fluid', 'in', 'left', 'pleural', 'space'), 2),
 (('in', 'left', 'pleural', 'space', '<\\s>'), 2),
 (('<s>', 'the', 'left', 'costophrenic', 'angle'), 1),
 (('the', 'left', 'costophrenic', 'angle', 'is'), 1),
 (('left', 'costophrenic', 'angle', 'is', 'still'), 1),
 (('costophrenic', 'angle', 'is', 'still', 'clear'), 1),
 (('angle', 'is', 'still', 'clear', '<\\s>'), 1),
 (('<s>', 'the', 'thickening', 'at', 'right'), 1),
 (('the', 'thickening', 'at', 'right', 'costophrenic'), 1),
 ((

In [441]:
pleural_sixgramk = createPatternNgrams(pleural_report, 6, pleural_keyword)
list(pleural_sixgramk.items())[:20]

[(('<s>', 'both', 'costophrenic', 'sulci', 'appear', 'unremarkable'), 8),
 (('both', 'costophrenic', 'sulci', 'appear', 'unremarkable', '<\\s>'), 8),
 (('<s>', 'loss', 'of', 'costophrenic', 'angle', 'with'), 2),
 (('loss', 'of', 'costophrenic', 'angle', 'with', 'fluid'), 2),
 (('of', 'costophrenic', 'angle', 'with', 'fluid', 'in'), 2),
 (('costophrenic', 'angle', 'with', 'fluid', 'in', 'left'), 2),
 (('angle', 'with', 'fluid', 'in', 'left', 'pleural'), 2),
 (('with', 'fluid', 'in', 'left', 'pleural', 'space'), 2),
 (('fluid', 'in', 'left', 'pleural', 'space', '<\\s>'), 2),
 (('<s>', 'the', 'left', 'costophrenic', 'angle', 'is'), 1),
 (('the', 'left', 'costophrenic', 'angle', 'is', 'still'), 1),
 (('left', 'costophrenic', 'angle', 'is', 'still', 'clear'), 1),
 (('costophrenic', 'angle', 'is', 'still', 'clear', '<\\s>'), 1),
 (('<s>', 'the', 'thickening', 'at', 'right', 'costophrenic'), 1),
 (('the', 'thickening', 'at', 'right', 'costophrenic', 'angle'), 1),
 (('thickening', 'at', 'right

In [442]:
pleural_keyword_2 = r'pleural effusion|costophrenic|effusion'

In [443]:
pleural_bigramk_2 = createPatternNgrams(pleural_report, 2, pleural_keyword_2)
list(pleural_bigramk_2.items())[:20]

[(('plerual', 'effusion'), 211),
 (('<s>', 'neither'), 192),
 (('effusion', 'nor'), 192),
 (('nor', 'pneumothorax'), 192),
 (('pneumothorax', '<\\s>'), 192),
 (('neither', 'plerual'), 191),
 (('change', 'of'), 105),
 (('no', 'change'), 81),
 (('effusion', '<\\s>'), 80),
 (('<s>', 'no'), 78),
 (('left', 'effusion'), 30),
 (('of', 'the'), 27),
 (('there', 'is'), 25),
 (('no', 'significant'), 24),
 (('effusion', 'is'), 24),
 (('significant', 'change'), 23),
 (('right', 'effusion'), 23),
 (('<s>', 'there'), 23),
 (('is', 'no'), 23),
 (('of', 'left'), 23)]

In [444]:
pleural_trigramk_2 = createPatternNgrams(pleural_report, 3, pleural_keyword_2)
list(pleural_trigramk_2.items())[:20]

[(('effusion', 'nor', 'pneumothorax'), 192),
 (('<s>', 'neither', 'plerual'), 191),
 (('neither', 'plerual', 'effusion'), 191),
 (('plerual', 'effusion', 'nor'), 191),
 (('nor', 'pneumothorax', '<\\s>'), 191),
 (('no', 'change', 'of'), 79),
 (('<s>', 'no', 'change'), 43),
 (('there', 'is', 'no'), 23),
 (('no', 'significant', 'change'), 22),
 (('<s>', 'there', 'is'), 22),
 (('significant', 'change', 'of'), 21),
 (('change', 'of', 'the'), 20),
 (('<s>', 'no', 'significant'), 17),
 (('is', 'no', 'change'), 17),
 (('change', 'of', 'left'), 17),
 (('left', 'effusion', '<\\s>'), 16),
 (('<s>', 'the', 'chest'), 13),
 (('the', 'chest', 'shows'), 13),
 (('chest', 'shows', 'no'), 13),
 (('effusion', 'compared', 'to'), 13)]

In [445]:
pleural_fourgramk_2 = createPatternNgrams(pleural_report, 4, pleural_keyword_2)
list(pleural_fourgramk_2.items())[:20]

[(('<s>', 'neither', 'plerual', 'effusion'), 191),
 (('neither', 'plerual', 'effusion', 'nor'), 191),
 (('plerual', 'effusion', 'nor', 'pneumothorax'), 191),
 (('effusion', 'nor', 'pneumothorax', '<\\s>'), 191),
 (('<s>', 'no', 'change', 'of'), 43),
 (('no', 'significant', 'change', 'of'), 21),
 (('<s>', 'there', 'is', 'no'), 21),
 (('no', 'change', 'of', 'the'), 19),
 (('there', 'is', 'no', 'change'), 17),
 (('is', 'no', 'change', 'of'), 17),
 (('<s>', 'no', 'significant', 'change'), 16),
 (('no', 'change', 'of', 'left'), 16),
 (('<s>', 'the', 'chest', 'shows'), 13),
 (('the', 'chest', 'shows', 'no'), 13),
 (('chest', 'shows', 'no', 'change'), 11),
 (('shows', 'no', 'change', 'of'), 11),
 (('<s>', 'both', 'costophrenic', 'sulci'), 9),
 (('of', 'the', 'right', 'effusion'), 8),
 (('no', 'change', 'of', 'right'), 8),
 (('change', 'of', 'left', 'effusion'), 8)]

# Atelectasis

In [446]:
# define keyword
# atelec_keyword = r'atelec|collap|volume'
atelec_keyword = r'atelec'

### Ground truth 0

Prepare Data

In [447]:
# choose reports
atelec_df_0 = selectData(atelec_data, 'Atelectasis', 0.0)
atelec_df_0

,Reports,Report List,Atelectasis BERT Labeler,Atelectasis Inspectra Labeler,Atelectasis Ground Truth,Atelectasis Key Detect
48,"Chest:\nCompared with 6-5-2008, no chage of RM...","['<s> chest <\\s>', '<s> compared with 6 5 200...",1,0,0.0,"<re.Match object; span=(27, 76), match='no cha..."
412,CXR PA upright\n\nHistory : ---\n\nComparison ...,"['<s> cxr pa upright <\\s>', '<s> history <\\s...",1,0,0.0,"<re.Match object; span=(4, 74), match='no demo..."
473,CXR PA upright\n\nMinimal right lower lung op...,"['<s> cxr pa upright <\\s>', '<s> minimal righ...",1,0,0.0,"<re.Match object; span=(4, 80), match='no demo..."
498,"CHEST FILM PA, UPRIGHT VIEW\n\nCOMPARISON: 07/...","['<s> chest film pa upright view <\\s>', '<s> ...",0,1,0.0,"<re.Match object; span=(4, 55), match='no inte..."
509,CXR\n Known case CA lung S/P CMT.\n Signific...,"['<s> cxr <\\s>', '<s> known case ca lung s p ...",0,1,0.0,"<re.Match object; span=(65, 131), match='no me..."
549,"CHEST, PA UPRIGHT\nAs comparison to prior CXR ...","['<s> chest pa upright <\\s>', '<s> as compari...",1,0,0.0,"<re.Match object; span=(81, 158), match='no sh..."
617,"CXR PA upright\n\nHistory : ESRD, HT, chronic...","['<s> cxr pa upright <\\s>', '<s> history esrd...",1,0,0.0,"<re.Match object; span=(4, 74), match='no demo..."
621,CXR PA upright\n\n Compared with previous che...,"['<s> cxr pa upright <\\s>', '<s> compared wit...",1,0,0.0,"<re.Match object; span=(20, 78), match='no mil..."
624,CXR PA upright\n\n\nAs compared with prior stu...,"['<s> cxr pa upright <\\s>', '<s> as compared ...",1,0,0.0,"<re.Match object; span=(4, 44), match='no demo..."
643,Chest PA upright (06/08/2018)\n\nCompared to f...,"['<s> chest pa upright 06 08 2018 <\\s>', '<s>...",0,1,0.0,"<re.Match object; span=(4, 121), match='no int..."


In [448]:
# create report list
atelec_report_0 = createReportList(atelec_df_0)
atelec_report_0[0]

['<s> chest <\\s>',
 '<s> compared with 6 5 2008 no chage of rml infiltration and atelectasis <\\s>',
 '<s> normal cardiothoracic ratio <\\s>']

Find N-grams Pattern

In [449]:
atelec_bigram_0 = createPatternNgrams(atelec_report_0, 2, atelec_keyword)
list(atelec_bigram_0.items())[:20]

[(('<s>', 'no'), 28),
 (('atelectasis', '<\\s>'), 24),
 (('or', 'atelectasis'), 17),
 (('no', 'demonstrable'), 16),
 (('mass', 'or'), 16),
 (('pulmonary', 'infiltration'), 13),
 (('active', 'pulmonary'), 12),
 (('infiltration', 'mass'), 11),
 (('demonstrable', 'active'), 9),
 (('no', 'interval'), 8),
 (('change', 'of'), 8),
 (('interval', 'change'), 6),
 (('is', 'noted'), 5),
 (('subsegmental', 'atelectasis'), 5),
 (('atelectasis', 'at'), 5),
 (('demonstrable', 'mass'), 4),
 (('and', 'atelectasis'), 3),
 (('noted', '<\\s>'), 3),
 (('with', 'no'), 3),
 (('<s>', 'as'), 3)]

In [450]:
atelec_trigram_0 = createPatternNgrams(atelec_report_0, 3, atelec_keyword)
list(atelec_trigram_0.items())[:20]

[(('<s>', 'no', 'demonstrable'), 16),
 (('mass', 'or', 'atelectasis'), 16),
 (('or', 'atelectasis', '<\\s>'), 16),
 (('active', 'pulmonary', 'infiltration'), 12),
 (('pulmonary', 'infiltration', 'mass'), 11),
 (('infiltration', 'mass', 'or'), 11),
 (('no', 'demonstrable', 'active'), 9),
 (('demonstrable', 'active', 'pulmonary'), 9),
 (('<s>', 'no', 'interval'), 6),
 (('no', 'interval', 'change'), 6),
 (('interval', 'change', 'of'), 6),
 (('no', 'demonstrable', 'mass'), 4),
 (('demonstrable', 'mass', 'or'), 4),
 (('is', 'noted', '<\\s>'), 3),
 (('and', 'atelectasis', '<\\s>'), 2),
 (('no', 'demonstrable', 'other'), 2),
 (('demonstrable', 'other', 'active'), 2),
 (('other', 'active', 'pulmonary'), 2),
 (('change', 'of', 'rul'), 2),
 (('of', 'rul', 'atelectasis'), 2)]

### Ground truth 1

Prepare Data

In [451]:
# choose reports
atelec_df_1 = selectData(atelec_data, 'Atelectasis', 1.0)
atelec_df_1

,Reports,Report List,Atelectasis BERT Labeler,Atelectasis Inspectra Labeler,Atelectasis Ground Truth,Atelectasis Key Detect
4,CXR\n\nRUL ateletasis.\nFibrocalcific infiltra...,"['<s> cxr <\\s>', '<s> rul ateletasis <\\s>', ...",1,0,1.0,"<re.Match object; span=(4, 23), match='rul ate..."
7,"Chest PA upright;\n known case CA lung, post C...","['<s> chest pa upright <\\s>', '<s> known case...",1,0,1.0,"<re.Match object; span=(62, 81), match='lll at..."
23,"CHEST.;\n\nKnoWn case of Ewing's sarcoma,post ...","['<s> chest <\\s>', ""<s> known case of ewing's...",1,0,1.0,"<re.Match object; span=(0, 20), match='<s> ate..."
25,CHEST\n\nAs compared with previous film on 20/...,"['<s> chest <\\s>', '<s> as compared with prev...",0,1,1.0,"<re.Match object; span=(0, 67), match='<s> ate..."
65,Chest PA.\n Ateletasis of RUL.\n right pleur...,"['<s> chest pa <\\s>', '<s> ateletasis of rul ...",1,0,1.0,"<re.Match object; span=(0, 26), match='<s> ate..."
176,CXR (PA)\n\nThe LLL opacity with sign of LLL v...,"['<s> cxr pa <\\s>', '<s> the lll opacity with...",1,0,1.0,"<re.Match object; span=(122, 142), match='lll ..."
185,Chest PA upruight \n\nHistory: RUL atelectasis...,"['<s> chest pa upruight <\\s>', '<s> history r...",1,0,1.0,"<re.Match object; span=(12, 32), match='rul at..."
186,CXR PA upright\n\nNo demonstrable active pulmo...,"['<s> cxr pa upright <\\s>', '<s> no demonstra...",0,1,1.0,"<re.Match object; span=(0, 64), match='<s> ate..."
317,CXR(PA upright)\n\n Hx of endobronchial tumor...,"['<s> cxr pa upright <\\s>', '<s> hx of endobr...",1,0,1.0,"<re.Match object; span=(42, 80), match='lul at..."
433,CXR\n\nS/P sternotomy.\nResolving LLL atelecta...,"['<s> cxr <\\s>', '<s> s p sternotomy <\\s>', ...",1,0,1.0,"<re.Match object; span=(14, 34), match='lll at..."


In [452]:
# create report list
atelec_report_1 = createReportList(atelec_df_1)
atelec_report_1[0]

['<s> cxr <\\s>',
 '<s> rul ateletasis <\\s>',
 '<s> fibrocalcific infiltration at both upper lobe and rll <\\s>',
 '<s> cardiomegaly <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> bont thorax is i8ntact <\\s>']


Find N-grams Pattern

In [453]:
atelec_bigram_1 = createPatternNgrams(atelec_report_1, 2, atelec_keyword)
list(atelec_bigram_1.items())[:20]

[(('<s>', 'atelectasis'), 10),
 (('atelectasis', '<\\s>'), 6),
 (('atelectasis', 'at'), 5),
 (('atelectesis', '<\\s>'), 4),
 (('atelectasis', 'and'), 4),
 (('rul', 'atelectasis'), 4),
 (('<s>', 'resolving'), 4),
 (('resolving', 'of'), 4),
 (('of', 'rul'), 4),
 (('at', 'rul'), 4),
 (('rul', '<\\s>'), 4),
 (('is', 'seen'), 4),
 (('rll', 'atelectasis'), 4),
 (('the', 'left'), 3),
 (('lll', 'atelectesis'), 3),
 (('<s>', 'partial'), 3),
 (('partial', 'resolved'), 3),
 (('atelectasis', 'is'), 3),
 (('seen', '<\\s>'), 3),
 (('at', 'lll'), 2)]

In [454]:
atelec_trigram_1 = createPatternNgrams(atelec_report_1, 3, atelec_keyword)
list(atelec_trigram_1.items())[:20]

[(('<s>', 'atelectasis', 'at'), 5),
 (('resolving', 'of', 'rul'), 4),
 (('at', 'rul', '<\\s>'), 4),
 (('lll', 'atelectesis', '<\\s>'), 3),
 (('atelectasis', 'at', 'rul'), 3),
 (('<s>', 'partial', 'resolved'), 3),
 (('rll', 'atelectasis', 'is'), 3),
 (('is', 'seen', '<\\s>'), 3),
 (('<s>', 'atelectasis', 'and'), 2),
 (('<s>', 'resolving', 'lll'), 2),
 (('resolving', 'lll', 'atelectasis'), 2),
 (('lll', 'atelectasis', '<\\s>'), 2),
 (('<s>', 'resolving', 'of'), 2),
 (('of', 'rul', 'atelectasis'), 2),
 (('rul', 'atelectasis', 'as'), 2),
 (('atelectasis', 'as', 'well'), 2),
 (('as', 'well', 'as'), 2),
 (('well', 'as', 'resolving'), 2),
 (('as', 'resolving', 'of'), 2),
 (('of', 'rul', 'interstitial'), 2)]

## Ground truth -1

Prepare Data

In [455]:
# choose reports
atelec_df_u = selectData(atelec_data, 'Atelectasis', -1.0)
atelec_df_u

,Reports,Report List,Atelectasis BERT Labeler,Atelectasis Inspectra Labeler,Atelectasis Ground Truth,Atelectasis Key Detect
3,"CXR\n Compare with film on 6/5/05, less expan...","['<s> cxr <\\s>', '<s> compare with film on 6 ...",0,1,-1.0,"<re.Match object; span=(4, 46), match='no chan..."
5,CXR PA upright\nHistory of ovarian cyst. \n\nS...,"['<s> cxr pa upright <\\s>', '<s> history of o...",0,1,-1.0,"<re.Match object; span=(10, 56), match='plate ..."
6,CXR PA upright\n By compared with previous ch...,"['<s> cxr pa upright <\\s>', '<s> by compared ...",0,1,-1.0,"<re.Match object; span=(18, 72), match='some a..."
9,CHEST\n\nPlate atelecsis at Rt lower lobe.\nFi...,"['<s> chest <\\s>', '<s> plate atelecsis at rt...",1,0,-1.0,"<re.Match object; span=(4, 41), match='plate a..."
11,CHEST\n\nCompared to previous film on 28/07/20...,"['<s> chest <\\s>', '<s> compared to previous ...",0,1,-1.0,"<re.Match object; span=(4, 104), match='no sig..."
...,...,...,...,...,...,...
2158,CXR PA upright\n\nHistory : RA with 1st hypot...,"['<s> cxr pa upright <\\s>', '<s> history ra w...",0,1,-1.0,"<re.Match object; span=(4, 62), match='no chan..."
2159,CXR(PA)\n\n Compare to CXR on 15/5/2015 and 1...,"['<s> cxr pa <\\s>', '<s> compare to cxr on 15...",0,1,-1.0,"<re.Match object; span=(4, 72), match='no chan..."
2162,Chest PA upright\n \n No pulmonary infiltrat...,"['<s> chest pa upright <\\s>', '<s> no pulmona...",1,0,-1.0,"<re.Match object; span=(4, 73), match='plate a..."
2163,CHEST FILM\n\nNo pulmonary infiltration is see...,"['<s> chest film <\\s>', '<s> no pulmonary inf...",1,0,-1.0,"<re.Match object; span=(4, 65), match='plate l..."


In [456]:
# create report list
atelec_report_u = createReportList(atelec_df_u)
atelec_report_u[0]

['<s> cxr <\\s>',
 '<s> compare with film on 6 5 05 less expansion of bilateral lungs are noted and possibly causing more prominent of interstitial thickening at lll <\\s>',
 '<s> no change of plate atelectasis at rll <\\s>',
 '<s> no cardiomegaly <\\s>',
 '<s> mildly prominent aorta is noted <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> intact bony thorax <\\s>']

Find N-grams Pattern

In [457]:
atelec_bigram_u = createPatternNgrams(atelec_report_u, 2, atelec_keyword)
list(atelec_bigram_u.items())[:20]

[(('atelectasis', 'at'), 644),
 (('change', 'of'), 514),
 (('<s>', 'no'), 476),
 (('plate', 'atelectasis'), 465),
 (('no', 'change'), 395),
 (('<s>', 'plate'), 222),
 (('at', 'left'), 221),
 (('of', 'plate'), 212),
 (('at', 'lll'), 193),
 (('no', 'significant'), 152),
 (('at', 'right'), 143),
 (('lung', '<\\s>'), 140),
 (('significant', 'change'), 124),
 (('lll', '<\\s>'), 110),
 (('plate', 'like'), 110),
 (('changed', 'of'), 109),
 (('like', 'atelectasis'), 108),
 (('lower', 'lung'), 107),
 (('of', 'atelectasis'), 95),
 (('noted', '<\\s>'), 92)]

In [458]:
atelec_trigram_u = createPatternNgrams(atelec_report_u, 3, atelec_keyword)
list(atelec_trigram_u.items())[:20]

[(('plate', 'atelectasis', 'at'), 392),
 (('no', 'change', 'of'), 385),
 (('<s>', 'no', 'change'), 276),
 (('atelectasis', 'at', 'left'), 199),
 (('of', 'plate', 'atelectasis'), 196),
 (('atelectasis', 'at', 'lll'), 172),
 (('change', 'of', 'plate'), 156),
 (('<s>', 'plate', 'atelectasis'), 125),
 (('no', 'significant', 'change'), 124),
 (('significant', 'change', 'of'), 119),
 (('plate', 'like', 'atelectasis'), 108),
 (('<s>', 'no', 'significant'), 105),
 (('atelectasis', 'at', 'right'), 100),
 (('like', 'atelectasis', 'at'), 99),
 (('at', 'lll', '<\\s>'), 97),
 (('no', 'changed', 'of'), 87),
 (('is', 'noted', '<\\s>'), 81),
 (('change', 'of', 'atelectasis'), 80),
 (('<s>', 'plate', 'like'), 76),
 (('at', 'left', 'lower'), 74)]

## Unlabel

Prepare Data

In [459]:
# choose reports
atelec_df = selectData(atelec_data, 'Atelectasis', None)
atelec_df

,Reports,Report List,Atelectasis BERT Labeler,Atelectasis Inspectra Labeler,Atelectasis Ground Truth,Atelectasis Key Detect
0,CXR\n Suboptimal inspiration is noted.\n Inc...,"['<s> cxr <\\s>', '<s> suboptimal inspiration ...",1,0,NaN,NaN
1,Chest film\nReticulonodular infiltration both ...,"['<s> chest film <\\s>', '<s> reticulonodular ...",1,0,NaN,NaN
2,CXR PA upright\nFibropathy and atelectasis at ...,"['<s> cxr pa upright <\\s>', '<s> fibropathy a...",0,1,NaN,NaN
8,CXR \n\nHistory: none\nComparison: 31.5.2005\...,"['<s> cxr <\\s>', '<s> history none <\\s>', '<...",1,0,NaN,NaN
10,CXR\n\nCardiomegaly with increase pulmonary va...,"['<s> cxr <\\s>', '<s> cardiomegaly with incre...",1,0,NaN,NaN
...,...,...,...,...,...,...
2164,Chest PA upright\n Compare to prior study date...,"['<s> chest pa upright <\\s>', '<s> compare to...",1,0,NaN,NaN
2165,CXR PA UPRIGHT\nFINDINGS:\nIncrease in amount ...,"['<s> cxr pa upright <\\s>', '<s> findings <\\...",1,0,NaN,NaN
2166,CXR PA UPRIGHT\n\nFINDINGS:\nFurther increase ...,"['<s> cxr pa upright <\\s>', '<s> findings <\\...",1,0,NaN,NaN
2168,CXR PA\n\nCompared study: 18/04/2018\nSuspicio...,"['<s> cxr pa <\\s>', '<s> compared study 18 04...",1,0,NaN,NaN


In [460]:
# create report list
atelec_report = createReportList(atelec_df)
atelec_report[0]

['<s> cxr <\\s>',
 '<s> suboptimal inspiration is noted <\\s>',
 '<s> increased lung marking at both basal lungs are noted and possibly due to collapse lung markings from not full inspiration <\\s>',
 '<s> no definite lung nifltration or nodule is seen <\\s>',
 '<s> heart size cannot be evaluate <\\s>']


Find N-grams Pattern

In [461]:
atelec_bigram = createPatternNgrams(atelec_report, 2)
list(atelec_bigram.items())[:20]

[(('<s>', 'no'), 1161),
 (('<s>', 'the'), 731),
 (('noted', '<\\s>'), 696),
 (('is', 'seen'), 617),
 (('seen', '<\\s>'), 611),
 (('is', 'noted'), 610),
 (('pleural', 'effusion'), 476),
 (('change', 'of'), 404),
 (('clear', '<\\s>'), 386),
 (('pulmonary', 'infiltration'), 343),
 (('both', 'costophrenic'), 337),
 (('effusion', '<\\s>'), 289),
 (('at', 'right'), 287),
 (('cardiomegaly', 'is'), 281),
 (('are', 'clear'), 276),
 (('degenerative', 'change'), 271),
 (('<s>', 'findings'), 271),
 (('costophrenic', 'angles'), 270),
 (('angles', 'are'), 268),
 (('<s>', 'both'), 265)]

In [462]:
atelec_trigram = createPatternNgrams(atelec_report, 3)
list(atelec_trigram.items())[:20]

[(('is', 'noted', '<\\s>'), 537),
 (('is', 'seen', '<\\s>'), 536),
 (('are', 'clear', '<\\s>'), 274),
 (('costophrenic', 'angles', 'are'), 261),
 (('pleural', 'effusion', '<\\s>'), 260),
 (('both', 'costophrenic', 'angles'), 249),
 (('<s>', 'no', 'cardiomegaly'), 240),
 (('<s>', 'both', 'costophrenic'), 203),
 (('is', 'intact', '<\\s>'), 176),
 (('bony', 'thorax', 'is'), 170),
 (('cardiomegaly', 'is', 'seen'), 168),
 (('pulmonary', 'infiltration', 'or'), 167),
 (('no', 'cardiomegaly', 'is'), 159),
 (('thorax', 'is', 'intact'), 157),
 (('calcified', 'aortic', 'knob'), 157),
 (('<s>', 'there', 'is'), 157),
 (('degenerative', 'change', 'of'), 154),
 (('<s>', 'degenerative', 'change'), 147),
 (('angles', 'are', 'clear'), 145),
 (('no', 'pleural', 'effusion'), 141)]

In [463]:
atelec_fourgram = createPatternNgrams(atelec_report, 4)
list(atelec_fourgram.items())[:20]

[(('both', 'costophrenic', 'angles', 'are'), 243),
 (('cardiomegaly', 'is', 'seen', '<\\s>'), 167),
 (('<s>', 'no', 'cardiomegaly', 'is'), 159),
 (('bony', 'thorax', 'is', 'intact'), 157),
 (('thorax', 'is', 'intact', '<\\s>'), 157),
 (('angles', 'are', 'clear', '<\\s>'), 144),
 (('costophrenic', 'angles', 'are', 'clear'), 139),
 (('<s>', 'no', 'pleural', 'effusion'), 138),
 (('<s>', 'degenerative', 'change', 'of'), 138),
 (('<s>', 'no', 'blunting', 'of'), 134),
 (('<s>', 'calcified', 'aortic', 'knob'), 132),
 (('is', 'not', 'enlarged', '<\\s>'), 129),
 (('no', 'blunting', 'of', 'the'), 128),
 (('visualized', 'bony', 'structures', 'are'), 123),
 (('<s>', 'both', 'costophrenic', 'angles'), 121),
 (('<s>', 'the', 'visualized', 'bony'), 121),
 (('the', 'visualized', 'bony', 'structures'), 120),
 (('heart', 'is', 'not', 'enlarged'), 119),
 (('calcified', 'aortic', 'knob', 'is'), 114),
 (('pulmonary', 'infiltration', 'or', 'nodule'), 110)]

In [464]:
atelec_bigramk = createPatternNgrams(atelec_report, 2, atelec_keyword)
list(atelec_bigramk.items())[:20]

[(('atelectasis', 'at'), 23),
 (('subsegmental', 'atelectasis'), 15),
 (('at', 'rul'), 13),
 (('and', 'atelectasis'), 9),
 (('at', 'right'), 9),
 (('at', 'lul'), 8),
 (('<s>', 'prior'), 8),
 (('of', 'right'), 8),
 (('rul', '<\\s>'), 7),
 (('lul', '<\\s>'), 7),
 (('is', 'absent'), 7),
 (('absent', '<\\s>'), 7),
 (('atelectasis', '<\\s>'), 7),
 (('atelectesis', '<\\s>'), 6),
 (('atelectasis', 'of'), 5),
 (('<s>', 'unchanged'), 5),
 (('at', 'rll'), 5),
 (('prior', 'seen'), 5),
 (('at', 'lll'), 5),
 (('rml', 'atelectasis'), 5)]

In [465]:
atelec_trigramk = createPatternNgrams(atelec_report, 3, atelec_keyword)
list(atelec_trigramk.items())[:20]

[(('subsegmental', 'atelectasis', 'at'), 12),
 (('atelectasis', 'at', 'rul'), 9),
 (('and', 'atelectasis', 'at'), 8),
 (('at', 'lul', '<\\s>'), 7),
 (('is', 'absent', '<\\s>'), 7),
 (('at', 'rul', '<\\s>'), 6),
 (('atelectasis', 'at', 'rll'), 5),
 (('<s>', 'prior', 'seen'), 5),
 (('at', 'rul', 'and'), 4),
 (('<s>', 'subsegmental', 'atelectasis'), 4),
 (('atelectasis', 'at', 'lll'), 4),
 (('<s>', 'no', 'change'), 4),
 (('no', 'change', 'of'), 4),
 (('<s>', 'unchanged', 'partially'), 3),
 (('unchanged', 'partially', 'atelecatis'), 3),
 (('partially', 'atelecatis', 'at'), 3),
 (('atelecatis', 'at', 'right'), 3),
 (('at', 'right', 'upper'), 3),
 (('right', 'upper', 'lung'), 3),
 (('upper', 'lung', 'and'), 3)]

In [466]:
atelec_fourgramk = createPatternNgrams(atelec_report, 4, atelec_keyword)
list(atelec_fourgramk.items())[:20]

[(('and', 'atelectasis', 'at', 'rul'), 6),
 (('atelectasis', 'at', 'rul', '<\\s>'), 6),
 (('<s>', 'subsegmental', 'atelectasis', 'at'), 4),
 (('subsegmental', 'atelectasis', 'at', 'rll'), 4),
 (('<s>', 'no', 'change', 'of'), 4),
 (('<s>', 'unchanged', 'partially', 'atelecatis'), 3),
 (('unchanged', 'partially', 'atelecatis', 'at'), 3),
 (('partially', 'atelecatis', 'at', 'right'), 3),
 (('atelecatis', 'at', 'right', 'upper'), 3),
 (('at', 'right', 'upper', 'lung'), 3),
 (('right', 'upper', 'lung', 'and'), 3),
 (('upper', 'lung', 'and', 'few'), 3),
 (('lung', 'and', 'few', 'nodule'), 3),
 (('and', 'few', 'nodule', 'shadow'), 3),
 (('few', 'nodule', 'shadow', 'at'), 3),
 (('nodule', 'shadow', 'at', 'lul'), 3),
 (('shadow', 'at', 'lul', '<\\s>'), 3),
 (('atelectasis', 'at', 'rll', 'is'), 3),
 (('subsegmental', 'atelectasis', 'at', 'lll'), 3),
 (('mass', 'and', 'atelectasis', 'at'), 2)]

In [467]:
atelec_fivegramk = createPatternNgrams(atelec_report, 5, atelec_keyword)
list(atelec_fivegramk.items())[:20]

[(('and', 'atelectasis', 'at', 'rul', '<\\s>'), 4),
 (('<s>', 'unchanged', 'partially', 'atelecatis', 'at'), 3),
 (('unchanged', 'partially', 'atelecatis', 'at', 'right'), 3),
 (('partially', 'atelecatis', 'at', 'right', 'upper'), 3),
 (('atelecatis', 'at', 'right', 'upper', 'lung'), 3),
 (('at', 'right', 'upper', 'lung', 'and'), 3),
 (('right', 'upper', 'lung', 'and', 'few'), 3),
 (('upper', 'lung', 'and', 'few', 'nodule'), 3),
 (('lung', 'and', 'few', 'nodule', 'shadow'), 3),
 (('and', 'few', 'nodule', 'shadow', 'at'), 3),
 (('few', 'nodule', 'shadow', 'at', 'lul'), 3),
 (('nodule', 'shadow', 'at', 'lul', '<\\s>'), 3),
 (('subsegmental', 'atelectasis', 'at', 'rll', 'is'), 3),
 (('mass', 'and', 'atelectasis', 'at', 'rul'), 2),
 (('and', 'atelectasis', 'at', 'rul', 'and'), 2),
 (('atelectasis', 'at', 'rll', 'is', 'absent'), 2),
 (('at', 'rll', 'is', 'absent', '<\\s>'), 2),
 (('subsegmental', 'atelectasis', 'at', 'lll', 'base'), 2),
 (('atelectasis', 'at', 'lll', 'base', 'is'), 2),
 (('

In [468]:
atelec_sixgramk = createPatternNgrams(atelec_report, 6, atelec_keyword)
list(atelec_sixgramk.items())[:20]

[(('<s>', 'unchanged', 'partially', 'atelecatis', 'at', 'right'), 3),
 (('unchanged', 'partially', 'atelecatis', 'at', 'right', 'upper'), 3),
 (('partially', 'atelecatis', 'at', 'right', 'upper', 'lung'), 3),
 (('atelecatis', 'at', 'right', 'upper', 'lung', 'and'), 3),
 (('at', 'right', 'upper', 'lung', 'and', 'few'), 3),
 (('right', 'upper', 'lung', 'and', 'few', 'nodule'), 3),
 (('upper', 'lung', 'and', 'few', 'nodule', 'shadow'), 3),
 (('lung', 'and', 'few', 'nodule', 'shadow', 'at'), 3),
 (('and', 'few', 'nodule', 'shadow', 'at', 'lul'), 3),
 (('few', 'nodule', 'shadow', 'at', 'lul', '<\\s>'), 3),
 (('mass', 'and', 'atelectasis', 'at', 'rul', '<\\s>'), 2),
 (('subsegmental', 'atelectasis', 'at', 'rll', 'is', 'absent'), 2),
 (('atelectasis', 'at', 'rll', 'is', 'absent', '<\\s>'), 2),
 (('subsegmental', 'atelectasis', 'at', 'lll', 'base', 'is'), 2),
 (('platatelectasis', 'at', 'right', 'lower', 'lung', 'with'), 2),
 (('at', 'right', 'lower', 'lung', 'with', 'minimal'), 2),
 (('right'

# Lung Lesion

In [469]:
# define keyword
lesion_keyword = r'nodule|mass'

## Ground truth 0

Prepare Data

In [470]:
# choose reports
lesion_df_0 = selectData(lesion_data, 'Lung Lesion', 0.0)
lesion_df_0

,Reports,Report List,Lung Lesion BERT Labeler,Lung Lesion Inspectra Labeler,Lung Lesion Ground Truth,Lung Lesion Key Detect
8,CXR\n\nHistory of goiter\n\nMild hyperinflatio...,"['<s> cxr <\\s>', '<s> history of goiter <\\s>...",1,0,0.0,"<re.Match object; span=(4, 23), match='no pulm..."
70,Chest PA upright\n Known case CA breast S/P CM...,"['<s> chest pa upright <\\s>', '<s> known case...",1,0,0.0,"<re.Match object; span=(4, 23), match='no pulm..."
72,CXR (PA upright)\nHx: Lt.parotidf tumor. Preo...,"['<s> cxr pa upright <\\s>', '<s> hx lt.paroti...",1,0,0.0,"<re.Match object; span=(4, 49), match='no defi..."
100,Chest film\nReticulonodular infiltration both ...,"['<s> chest film <\\s>', '<s> reticulonodular ...",1,0,0.0,"<re.Match object; span=(4, 20), match='no defi..."
119,Chest PA upright;\n compared to filmon 13/5/20...,"['<s> chest pa upright <\\s>', '<s> compared t...",0,1,0.0,"<re.Match object; span=(4, 33), match='no new ..."
...,...,...,...,...,...,...
34201,CXR PA upright\n\n There is no definite pulmo...,"['<s> cxr pa upright <\\s>', '<s> there is no ...",0,1,0.0,"<re.Match object; span=(13, 57), match='no def..."
34227,CXR PA upright\n\nHistory : ESRD \n\nCompari...,"['<s> cxr pa upright <\\s>', '<s> history esrd...",0,1,0.0,"<re.Match object; span=(4, 54), match='no demo..."
34267,CXR PA UPRIGHT \n\nFINDINGS:\n No pulmonary i...,"['<s> cxr pa upright <\\s>', '<s> findings <\\...",0,1,0.0,"<re.Match object; span=(4, 39), match='no pulm..."
34294,CXR\n\nCompared to prior film on 9 december 17...,"['<s> cxr <\\s>', '<s> compared to prior film ...",0,1,0.0,"<re.Match object; span=(57, 85), match='no def..."


In [471]:
# create report list
lesion_report_0 = createReportList(lesion_df_0)
lesion_report_0[0]

['<s> cxr <\\s>',
 '<s> history of goiter <\\s>',
 '<s> mild hyperinflation is noted bilaterally <\\s>',
 '<s> blunting of right costophrenic angle could be small pleural effusion or pleural thickening <\\s>',
 '<s> calcification at right lower lobe close to lateral chest wall <\\s>',
 '<s> interstitial thickening both lungs <\\s>',
 '<s> no pulmonary nodule is demonstrated <\\s>']

Find N-grams Pattern

In [472]:
lesion_bigram_0 = createPatternNgrams(lesion_report_0, 2, lesion_keyword)
list(lesion_bigram_0.items())[:20]

[(('<s>', 'no'), 998),
 (('pulmonary', 'infiltration'), 805),
 (('infiltration', 'or'), 494),
 (('or', 'nodule'), 421),
 (('active', 'pulmonary'), 421),
 (('no', 'demonstrable'), 363),
 (('mass', 'or'), 345),
 (('or', 'atelectasis'), 334),
 (('atelectasis', '<\\s>'), 330),
 (('is', 'noted'), 311),
 (('noted', '<\\s>'), 309),
 (('infiltration', 'mass'), 308),
 (('nodule', 'is'), 298),
 (('demonstrable', 'active'), 291),
 (('nodule', 'at'), 262),
 (('no', 'definite'), 236),
 (('no', 'detectable'), 228),
 (('nodule', '<\\s>'), 214),
 (('change', 'of'), 212),
 (('detectable', 'pulmonary'), 170)]

In [473]:
lesion_trigram_0 = createPatternNgrams(lesion_report_0, 3, lesion_keyword)
list(lesion_trigram_0.items())[:20]

[(('pulmonary', 'infiltration', 'or'), 427),
 (('active', 'pulmonary', 'infiltration'), 414),
 (('infiltration', 'or', 'nodule'), 411),
 (('<s>', 'no', 'demonstrable'), 360),
 (('mass', 'or', 'atelectasis'), 334),
 (('or', 'atelectasis', '<\\s>'), 329),
 (('pulmonary', 'infiltration', 'mass'), 308),
 (('infiltration', 'mass', 'or'), 295),
 (('is', 'noted', '<\\s>'), 292),
 (('demonstrable', 'active', 'pulmonary'), 291),
 (('no', 'demonstrable', 'active'), 290),
 (('or', 'nodule', 'is'), 247),
 (('nodule', 'is', 'noted'), 191),
 (('no', 'detectable', 'pulmonary'), 169),
 (('detectable', 'pulmonary', 'infiltration'), 157),
 (('<s>', 'findings', 'no'), 152),
 (('findings', 'no', 'detectable'), 150),
 (('no', 'definite', 'pulmonary'), 132),
 (('or', 'nodule', '<\\s>'), 125),
 (('calcified', 'nodule', 'at'), 119)]

## Ground truth 1

Prepare Data

In [474]:
# choose reports
lesion_df_1 = selectData(lesion_data, 'Lung Lesion', 1.0)
lesion_df_1

,Reports,Report List,Lung Lesion BERT Labeler,Lung Lesion Inspectra Labeler,Lung Lesion Ground Truth,Lung Lesion Key Detect
170,Chest PA upright;\n Advanced CA breast with lu...,"['<s> chest pa upright <\\s>', '<s> advanced c...",1,0,1.0,"<re.Match object; span=(27, 37), match='lll no..."
289,Chest PA urpight;\n known case CA lung S/P ch...,"['<s> chest pa urpight <\\s>', '<s> known case...",1,0,1.0,"<re.Match object; span=(58, 66), match='rul ma..."
290,Chest PA upright \nknown case small cell lung ...,"['<s> chest pa upright <\\s>', '<s> known case...",1,0,1.0,"<re.Match object; span=(23, 31), match='lul ma..."
306,CXR\n Hx of CA lung S/P CMT.\n Ill-defined...,"['<s> cxr <\\s>', '<s> hx of ca lung s p cmt <...",0,1,1.0,"<re.Match object; span=(8, 71), match='defined..."
332,CXR PA UPRIGHT\n\n A 67 years old woman w...,"['<s> cxr pa upright <\\s>', '<s> a 67 years o...",1,0,1.0,"<re.Match object; span=(4, 14), match='lul nod..."
...,...,...,...,...,...,...
33359,CXR PA\n\nThere is increased amount of left ma...,"['<s> cxr pa <\\s>', '<s> there is increased a...",1,0,1.0,"<re.Match object; span=(33, 42), match='left m..."
33368,CXR : PA\n\nHistory: RUL nodule S/P surgery\nS...,"['<s> cxr pa <\\s>', '<s> history rul nodule s...",1,0,1.0,"<re.Match object; span=(12, 22), match='rul no..."
33584,CXR PA upright\n\nNo demonstrable active pulmo...,"['<s> cxr pa upright <\\s>', '<s> no demonstra...",0,1,1.0,"<re.Match object; span=(33, 41), match='rll ma..."
33585,CXR : PA\n\nHistory: RUL nodule S/P surgery\nN...,"['<s> cxr pa <\\s>', '<s> history rul nodule s...",1,0,1.0,"<re.Match object; span=(12, 22), match='rul no..."


In [475]:
# create report list
lesion_report_1 = createReportList(lesion_df_1)
lesion_report_1[0]

['<s> chest pa upright <\\s>',
 '<s> advanced ca breast with lung and bone metastasis <\\s>',
 '<s> no striking changed of reticulonoduular infiltration in lt.lung <\\s>',
 '<s> there is not well seen lll nodule <\\s>',
 '<s> rtlung is relative clear <\\s>',
 '<s> heart is mild enlared <\\s>',
 '<s> ds <\\s>']

Find N-grams Pattern

In [476]:
lesion_bigram_1 = createPatternNgrams(lesion_report_1, 2, lesion_keyword)
list(lesion_bigram_1.items())[:20]

[(('<s>', 'history'), 23),
 (('pleural', 'effusion'), 19),
 (('nodule', 'at'), 18),
 (('<s>', 'no'), 17),
 (('rul', 'nodule'), 17),
 (('nodule', 'is'), 16),
 (('<s>', 'massive'), 15),
 (('at', 'left'), 14),
 (('nodule', '<\\s>'), 13),
 (('<s>', 'a'), 13),
 (('mass', '<\\s>'), 12),
 (('right', 'pleural'), 12),
 (('well', 'defined'), 12),
 (('there', 'is'), 11),
 (('is', 'noted'), 11),
 (('is', 'seen'), 11),
 (('effusion', '<\\s>'), 11),
 (('at', 'right'), 11),
 (('<s>', 'there'), 10),
 (('rul', 'mass'), 10)]

In [477]:
lesion_trigram_1 = createPatternNgrams(lesion_report_1, 3, lesion_keyword)
list(lesion_trigram_1.items())[:20]

[(('right', 'pleural', 'effusion'), 12),
 (('<s>', 'there', 'is'), 10),
 (('pleural', 'effusion', '<\\s>'), 9),
 (('<s>', 'massive', 'right'), 9),
 (('massive', 'right', 'pleural'), 9),
 (('nodule', 'is', 'seen'), 6),
 (('left', 'pleural', 'effusion'), 6),
 (('<s>', 'history', 'rul'), 6),
 (('is', 'seen', '<\\s>'), 6),
 (('this', 'study', '<\\s>'), 5),
 (('pleural', 'effusion', 'with'), 5),
 (('pleural', 'effusion', 'is'), 5),
 (('of', 'rul', 'nodule'), 5),
 (('at', 'right', 'lower'), 5),
 (('lower', 'lobe', '<\\s>'), 5),
 (('<s>', 'nodule', 'at'), 5),
 (('history', 'rul', 'nodule'), 5),
 (('nipple', 'shadow', '<\\s>'), 5),
 (('<s>', 'history', 'of'), 4),
 (('<s>', 'no', 'definite'), 4)]

## Ground truth -1

Prepare Data

In [478]:
# choose reports
lesion_df_u = selectData(lesion_data, 'Lung Lesion', -1.0)
lesion_df_u

,Reports,Report List,Lung Lesion BERT Labeler,Lung Lesion Inspectra Labeler,Lung Lesion Ground Truth,Lung Lesion Key Detect
7,CXR\n\n No significant change of a RUL nodu...,"['<s> cxr <\\s>', '<s> no significant change o...",0,1,-1.0,"<re.Match object; span=(4, 41), match='no sign..."
13,Chest;\n\n Small calcified nodule at right lo...,"['<s> chest <\\s>', '<s> small calcified nodul...",0,1,-1.0,"<re.Match object; span=(4, 26), match='small c..."
23,CXR\n\nHistory of CA rectum\n\nAs compared to ...,"['<s> cxr <\\s>', '<s> history of ca rectum <\...",0,1,-1.0,"<re.Match object; span=(59, 97), match='no cha..."
25,CXR PA upright\n The chest shows multiple cal...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,-1.0,"<re.Match object; span=(20, 44), match='multip..."
31,"CXR\n Compare with film on 21/4/05, no change...","['<s> cxr <\\s>', '<s> compare with film on 21...",1,0,-1.0,"<re.Match object; span=(33, 85), match='no cha..."
...,...,...,...,...,...,...
34319,CXR PA upright\n\nComparison to prior radiogra...,"['<s> cxr pa upright <\\s>', '<s> comparison t...",0,1,-1.0,"<re.Match object; span=(26, 41), match='multip..."
34328,Chest PA upright\n\n Overall no significant c...,"['<s> chest pa upright <\\s>', '<s> overall no...",0,1,-1.0,"<re.Match object; span=(12, 68), match='no sig..."
34331,CXR PA UPRIGHT\n\nThere is a small calcified n...,"['<s> cxr pa upright <\\s>', '<s> there is a s...",0,1,-1.0,"<re.Match object; span=(15, 37), match='small ..."
34336,CXR PA upright\n\n The chest shows no change ...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,-1.0,"<re.Match object; span=(20, 52), match='no cha..."


In [479]:
# create report list
lesion_report_u = createReportList(lesion_df_u)
lesion_report_u[0]

['<s> cxr <\\s>',
 '<s> no significant change of a rul nodule as compare with film on 5 2 05 <\\s>',
 '<s> rest lungs are clear <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> cardiomegaly with markedly dilated and calcified aorta suggested aneurysm <\\s>']

Find N-grams Pattern

In [480]:
lesion_bigram_u = createPatternNgrams(lesion_report_u, 2, lesion_keyword)
list(lesion_bigram_u.items())[:20]

[(('change', 'of'), 5735),
 (('<s>', 'no'), 3502),
 (('no', 'change'), 3293),
 (('nodule', 'at'), 2757),
 (('no', 'significant'), 2430),
 (('significant', 'change'), 2225),
 (('at', 'right'), 1844),
 (('nodules', 'at'), 1725),
 (('<s>', 'the'), 1683),
 (('the', 'chest'), 1498),
 (('chest', 'shows'), 1494),
 (('shows', 'no'), 1457),
 (('calcified', 'nodule'), 1321),
 (('at', 'left'), 1143),
 (('at', 'both'), 1138),
 (('of', 'multiple'), 1092),
 (('lung', '<\\s>'), 1085),
 (('of', 'a'), 1047),
 (('both', 'lungs'), 974),
 (('at', 'rul'), 893)]

In [481]:
lesion_trigram_u = createPatternNgrams(lesion_report_u, 3, lesion_keyword)
list(lesion_trigram_u.items())[:20]

[(('no', 'change', 'of'), 3248),
 (('no', 'significant', 'change'), 2224),
 (('significant', 'change', 'of'), 2204),
 (('<s>', 'no', 'significant'), 1656),
 (('<s>', 'the', 'chest'), 1496),
 (('the', 'chest', 'shows'), 1494),
 (('chest', 'shows', 'no'), 1427),
 (('shows', 'no', 'change'), 1320),
 (('<s>', 'no', 'change'), 1193),
 (('calcified', 'nodule', 'at'), 1075),
 (('change', 'of', 'multiple'), 994),
 (('change', 'of', 'a'), 879),
 (('nodules', 'at', 'both'), 834),
 (('nodule', 'at', 'right'), 754),
 (('at', 'both', 'lungs'), 696),
 (('both', 'lungs', '<\\s>'), 693),
 (('small', 'calcified', 'nodule'), 654),
 (('calcific', 'nodule', 'at'), 597),
 (('there', 'is', 'no'), 543),
 (('<s>', 'there', 'is'), 542)]

## Unlabel

Prepare Data

In [482]:
# choose reports
lesion_df = selectData(lesion_data, 'Lung Lesion', None)
lesion_df

,Reports,Report List,Lung Lesion BERT Labeler,Lung Lesion Inspectra Labeler,Lung Lesion Ground Truth,Lung Lesion Key Detect
0,CXR (PA upright)\n\nReticulonodular infiltrati...,"['<s> cxr pa upright <\\s>', '<s> reticulonodu...",1,0,NaN,NaN
1,Chest:-\nPa chest study reveals fibronodular i...,"['<s> chest <\\s>', '<s> pa chest study reveal...",1,0,NaN,NaN
2,CHEST :\nP.A. upright view .\nFibronodular inf...,"['<s> chest <\\s>', '<s> p.a <\\s>', '<s> upri...",1,0,NaN,NaN
3,CXR\n No previous film to be compared.\n RLL...,"['<s> cxr <\\s>', '<s> no previous film to be ...",1,0,NaN,NaN
4,CXR PA upright\n The chest shows mild reticul...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",1,0,NaN,NaN
...,...,...,...,...,...,...
34343,CXR PA upright\n\n DIffuse reticulonodular in...,"['<s> cxr pa upright <\\s>', '<s> diffuse reti...",1,0,NaN,NaN
34344,CXR PA \n\nDiffuse reticulonodular infiltratio...,"['<s> cxr pa <\\s>', '<s> diffuse reticulonodu...",1,0,NaN,NaN
34345,Chest PA upright \n\nknown CA breast\n\nSurgic...,"['<s> chest pa upright <\\s>', '<s> known ca b...",1,0,NaN,NaN
34346,CXR(PA)\n \n Retilocalcified nodules of bot...,"['<s> cxr pa <\\s>', '<s> retilocalcified nodu...",0,1,NaN,NaN


In [483]:
# create report list
lesion_report = createReportList(lesion_df)
lesion_report[0]

['<s> cxr pa upright <\\s>',
 '<s> reticulonodular infiltration at both upper lobes <\\s>',
 '<s> the heart and great vessels appear normal <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> bony thorax is intact <\\s>',
 '<s> imp tb is considered <\\s>',
 '<s> clinical and previous film correlation are <\\s>',
 '<s> suggested <\\s>']

Find N-grams Pattern

In [484]:
lesion_bigram = createPatternNgrams(lesion_report, 2)
list(lesion_bigram.items())[:20]

[(('<s>', 'no'), 23518),
 (('bony', 'thorax'), 12661),
 (('pleural', 'effusion'), 11971),
 (('infiltration', 'at'), 11956),
 (('<s>', 'the'), 11188),
 (('intact', '<\\s>'), 10065),
 (('thorax', 'is'), 9596),
 (('reticulonodular', 'infiltration'), 9283),
 (('is', 'intact'), 9102),
 (('<s>', 'bony'), 8607),
 (('is', 'seen'), 8266),
 (('effusion', '<\\s>'), 8057),
 (('seen', '<\\s>'), 8012),
 (('noted', '<\\s>'), 7662),
 (('clear', '<\\s>'), 7477),
 (('no', 'pleural'), 7153),
 (('both', 'costophrenic'), 7096),
 (('<s>', 'both'), 6977),
 (('is', 'noted'), 6822),
 (('<s>', 'normal'), 6668)]

In [485]:
lesion_trigram = createPatternNgrams(lesion_report, 3)
list(lesion_trigram.items())[:20]

[(('bony', 'thorax', 'is'), 9578),
 (('is', 'intact', '<\\s>'), 9098),
 (('thorax', 'is', 'intact'), 8615),
 (('<s>', 'bony', 'thorax'), 7723),
 (('pleural', 'effusion', '<\\s>'), 7618),
 (('no', 'pleural', 'effusion'), 7118),
 (('is', 'seen', '<\\s>'), 7020),
 (('<s>', 'no', 'pleural'), 6747),
 (('reticulonodular', 'infiltration', 'at'), 6575),
 (('are', 'clear', '<\\s>'), 6186),
 (('<s>', 'both', 'costophrenic'), 6123),
 (('is', 'noted', '<\\s>'), 6013),
 (('<s>', 'no', 'cardiomegaly'), 5875),
 (('costophrenic', 'angles', 'are'), 4800),
 (('<s>', 'there', 'is'), 4541),
 (('both', 'costophrenic', 'angles'), 4284),
 (('angles', 'are', 'clear'), 4040),
 (('costophrenic', 'sulci', 'are'), 3505),
 (('pleural', 'effusion', 'is'), 3471),
 (('no', 'cardiomegaly', '<\\s>'), 3445)]

In [486]:
lesion_fourgram = createPatternNgrams(lesion_report, 4)
list(lesion_fourgram.items())[:20]

[(('thorax', 'is', 'intact', '<\\s>'), 8611),
 (('bony', 'thorax', 'is', 'intact'), 8607),
 (('<s>', 'bony', 'thorax', 'is'), 7191),
 (('<s>', 'no', 'pleural', 'effusion'), 6715),
 (('no', 'pleural', 'effusion', '<\\s>'), 5130),
 (('both', 'costophrenic', 'angles', 'are'), 4124),
 (('angles', 'are', 'clear', '<\\s>'), 4031),
 (('costophrenic', 'angles', 'are', 'clear'), 3934),
 (('<s>', 'both', 'costophrenic', 'angles'), 3419),
 (('<s>', 'normal', 'cardiothoracic', 'ratio'), 2984),
 (('are', 'not', 'remarkable', '<\\s>'), 2931),
 (('<s>', 'no', 'cardiomegaly', '<\\s>'), 2809),
 (('normal', 'cardiothoracic', 'ratio', '<\\s>'), 2776),
 (('<s>', 'both', 'costophrenic', 'sulci'), 2680),
 (('<s>', 'no', 'cardiomegaly', 'is'), 2610),
 (('is', 'not', 'enlarged', '<\\s>'), 2434),
 (('<s>', 'reticulonodular', 'infiltration', 'at'), 2400),
 (('<s>', 'the', 'bony', 'thorax'), 2218),
 (('the', 'bony', 'thorax', 'is'), 2194),
 (('sulci', 'are', 'not', 'remarkable'), 2100)]

In [487]:
lesion_bigramk = createPatternNgrams(lesion_report, 2, lesion_keyword)
list(lesion_bigramk.items())[:20]

[(('nodule', 'at'), 823),
 (('calcified', 'nodule'), 752),
 (('<s>', 'a'), 415),
 (('<s>', 'calcified'), 376),
 (('at', 'right'), 313),
 (('at', 'rul'), 302),
 (('granuloma', '<\\s>'), 282),
 (('nodules', 'at'), 275),
 (('calcific', 'nodule'), 273),
 (('is', 'seen'), 243),
 (('at', 'left'), 229),
 (('calcified', 'nodules'), 211),
 (('at', 'lul'), 187),
 (('calcified', 'granuloma'), 186),
 (('<s>', 'calcific'), 186),
 (('calcific', 'nodules'), 179),
 (('seen', 'at'), 156),
 (('a', 'calcified'), 153),
 (('<s>', 'there'), 153),
 (('at', 'rll'), 139)]

In [488]:
lesion_trigramk = createPatternNgrams(lesion_report, 3, lesion_keyword)
list(lesion_trigramk.items())[:20]

[(('calcified', 'nodule', 'at'), 567),
 (('<s>', 'calcified', 'nodule'), 288),
 (('nodule', 'at', 'rul'), 186),
 (('calcific', 'nodule', 'at'), 172),
 (('calcified', 'granuloma', '<\\s>'), 170),
 (('nodule', 'at', 'right'), 166),
 (('a', 'calcified', 'nodule'), 152),
 (('nodule', 'at', 'left'), 138),
 (('calcified', 'nodules', 'at'), 136),
 (('<s>', 'a', 'calcified'), 123),
 (('<s>', 'calcific', 'nodule'), 118),
 (('<s>', 'there', 'is'), 117),
 (('be', 'calcified', 'granuloma'), 111),
 (('calcific', 'nodules', 'at'), 107),
 (('nodule', 'is', 'seen'), 104),
 (('nodule', 'at', 'lul'), 104),
 (('is', 'seen', 'at'), 103),
 (('is', 'seen', '<\\s>'), 93),
 (('at', 'rul', '<\\s>'), 89),
 (('nodule', 'at', 'rll'), 87)]

In [489]:
lesion_fourgramk = createPatternNgrams(lesion_report, 4, lesion_keyword)
list(lesion_fourgramk.items())[:20]

[(('<s>', 'calcified', 'nodule', 'at'), 202),
 (('calcified', 'nodule', 'at', 'rul'), 131),
 (('<s>', 'a', 'calcified', 'nodule'), 122),
 (('calcified', 'nodule', 'at', 'right'), 116),
 (('a', 'calcified', 'nodule', 'at'), 111),
 (('be', 'calcified', 'granuloma', '<\\s>'), 104),
 (('nodule', 'is', 'seen', 'at'), 102),
 (('calcified', 'nodule', 'at', 'left'), 90),
 (('which', 'may', 'be', 'calcified'), 73),
 (('<s>', 'calcific', 'nodule', 'at'), 73),
 (('calcified', 'nodule', 'at', 'lul'), 73),
 (('may', 'be', 'calcified', 'granuloma'), 68),
 (('calcified', 'nodule', 'is', 'seen'), 63),
 (('tiny', 'calcified', 'nodule', 'at'), 63),
 (('calcified', 'nodule', 'at', 'rll'), 59),
 (('nodule', 'at', 'rul', '<\\s>'), 59),
 (('<s>', 'calcified', 'nodule', 'is'), 56),
 (('<s>', 'a', 'calcific', 'nodule'), 56),
 (('calcified', 'nodule', 'at', 'lll'), 56),
 (('nodule', 'at', 'rul', 'is'), 54)]

In [490]:
lesion_fivegramk = createPatternNgrams(lesion_report, 5, lesion_keyword)
list(lesion_fivegramk.items())[:20]

[(('<s>', 'a', 'calcified', 'nodule', 'at'), 86),
 (('may', 'be', 'calcified', 'granuloma', '<\\s>'), 67),
 (('which', 'may', 'be', 'calcified', 'granuloma'), 64),
 (('calcified', 'nodule', 'is', 'seen', 'at'), 62),
 (('<s>', 'calcified', 'nodule', 'at', 'rul'), 55),
 (('<s>', 'calcified', 'nodule', 'is', 'seen'), 54),
 (('calcified', 'nodule', 'at', 'rul', '<\\s>'), 43),
 (('calcified', 'nodule', 'at', 'rul', 'is'), 40),
 (('a', 'tiny', 'calcified', 'nodule', 'at'), 39),
 (('could', 'be', 'calcified', 'granuloma', '<\\s>'), 35),
 (('<s>', 'a', 'tiny', 'calcified', 'nodule'), 35),
 (('<s>', 'calcified', 'nodule', 'at', 'left'), 32),
 (('nodule', 'at', 'right', 'lower', 'lung'), 31),
 (('<s>', 'calcified', 'nodule', 'at', 'right'), 31),
 (('calcified', 'nodule', 'at', 'right', 'lower'), 30),
 (('<s>', 'calcified', 'nodule', 'at', 'lul'), 29),
 (('calcified', 'nodule', 'at', 'lul', 'is'), 29),
 (('nodule', 'at', 'right', 'mid', 'lung'), 28),
 (('calcified', 'nodule', 'at', 'lll', 'is'), 

In [491]:
lesion_sixgramk = createPatternNgrams(lesion_report, 6, lesion_keyword)
list(lesion_sixgramk.items())[:20]

[(('which', 'may', 'be', 'calcified', 'granuloma', '<\\s>'), 64),
 (('<s>', 'calcified', 'nodule', 'is', 'seen', 'at'), 54),
 (('<s>', 'a', 'tiny', 'calcified', 'nodule', 'at'), 31),
 (('calcified', 'nodule', 'at', 'right', 'lower', 'lung'), 27),
 (('<s>', 'there', 'is', 'calcified', 'nodule', 'at'), 25),
 (('<s>', 'calcified', 'nodule', 'at', 'rul', 'is'), 23),
 (('<s>', 'a', 'few', 'calcific', 'nodules', 'at'), 22),
 (('<s>', 'a', 'calcified', 'nodule', 'at', 'right'), 22),
 (('calcified', 'nodule', 'at', 'right', 'mid', 'lung'), 21),
 (('<s>', 'calcified', 'nodules', 'are', 'seen', 'at'), 19),
 (('<s>', 'a', 'calcified', 'nodule', 'at', 'rul'), 18),
 (('seen', 'at', 'rul', 'which', 'may', 'be'), 17),
 (('at', 'rul', 'which', 'may', 'be', 'calcified'), 17),
 (('rul', 'which', 'may', 'be', 'calcified', 'granuloma'), 17),
 (('<s>', 'there', 'is', 'a', 'calcified', 'nodule'), 17),
 (('<s>', 'a', 'calcific', 'nodule', 'is', 'seen'), 16),
 (('calcified', 'nodule', 'at', 'left', 'mid', 'lu

# Inspectra Lung Opacity v1

In [492]:
# define keyword
infil_keyword = r'infil|densit|opaci|'
conso_keyword = 'consolidat|'
opaci_keyword = 'opaci|translu|air|mark|pattern|lung|reticul|scar|thicken|densit|patchy'
v1_keyword = infil_keyword + conso_keyword + opaci_keyword

## Ground truth 0

Prepare Data

In [493]:
# choose reports
v1_df_0 = selectData(v1_data, 'Inspectra Lung Opacity v1', 0.0)
v1_df_0

,Reports,Report List,Inspectra Lung Opacity v1 BERT Labeler,Inspectra Lung Opacity v1 Inspectra Labeler,Inspectra Lung Opacity v1 Ground Truth,Inspectra Lung Opacity v1 Key Detect
13,Chest film\n\nNo definite pulmonary infiltrati...,"['<s> chest film <\\s>', '<s> no definite pulm...",0,1,0.0,"<re.Match object; span=(4, 31), match='no defi..."
43,CXR (PA upright)\n\nNo definite pulmonary inf...,"['<s> cxr pa upright <\\s>', '<s> no definite ...",0,1,0.0,"<re.Match object; span=(4, 31), match='no defi..."
61,CXR PA upright\n The chest shows no definite ...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,0.0,"<re.Match object; span=(20, 47), match='no def..."
97,CXR \n\nHistory of chronic cough\n\nNo definit...,"['<s> cxr <\\s>', '<s> history of chronic coug...",0,1,0.0,"<re.Match object; span=(4, 31), match='no defi..."
101,CXR\n No pulmonary infiltration or nodule.\n ...,"['<s> cxr <\\s>', '<s> no pulmonary infiltrati...",0,1,0.0,"<re.Match object; span=(4, 22), match='no pulm..."
...,...,...,...,...,...,...
76632,CHEST\n\nNo cardiomegaly is noted.\nNo changhe...,"['<s> chest <\\s>', '<s> no cardiomegaly is no...",0,1,0.0,"<re.Match object; span=(4, 41), match='no chan..."
76645,Chest PA upright\n\nFindings:\nNo definite pul...,"['<s> chest pa upright <\\s>', '<s> findings <...",0,1,0.0,"<re.Match object; span=(4, 31), match='no defi..."
76649,Chest\nNo new active infiltration\nUnchanged i...,"['<s> chest <\\s>', '<s> no new active infiltr...",0,1,0.0,"<re.Match object; span=(4, 23), match='no new ..."
76651,CXR PA UPRIGHT\n\nFINDINGS : No detectable pu...,"['<s> cxr pa upright <\\s>', '<s> findings no ...",0,1,0.0,"<re.Match object; span=(13, 72), match='no det..."


In [494]:
# create report list
v1_report_0 = createReportList(v1_df_0)
v1_report_0[0]

['<s> chest film <\\s>',
 '<s> no definite pulmonary infiltration <\\s>',
 '<s> radiodensity line in rul likely skin fold <\\s>',
 '<s> cardiomegaly with dilated aorta <\\s>',
 '<s> right paratracheal density could be vascular shadow <\\s>',
 '<s> both costrophrenic angles are clear <\\s>',
 '<s> old fracture right 8th rib <\\s>']

Find N-grams Pattern

In [495]:
v1_bigram_0 = createPatternNgrams(v1_report_0, 2, v1_keyword)
list(v1_bigram_0.items())[:20]

[(('pulmonary', 'infiltration'), 30683),
 (('infiltration', 'or'), 15368),
 (('or', 'nodule'), 15123),
 (('infiltration', '<\\s>'), 14568),
 (('no', 'definite'), 14504),
 (('active', 'pulmonary'), 13993),
 (('definite', 'pulmonary'), 13711),
 (('there', 'is'), 13424),
 (('is', 'no'), 13063),
 (('<s>', 'there'), 13054),
 (('<s>', 'imp'), 12808),
 (('nodule', '<\\s>'), 12804),
 (('imp', 'no'), 12791),
 (('no', 'active'), 11770),
 (('noted', '<\\s>'), 4259),
 (('is', 'noted'), 3866),
 (('<s>', 'no'), 3760),
 (('<s>', 'findings'), 3094),
 (('infiltration', 'at'), 2980),
 (('no', 'detectable'), 2772)]

In [496]:
v1_trigram_0 = createPatternNgrams(v1_report_0, 3, v1_keyword)
list(v1_trigram_0.items())[:20]

[(('pulmonary', 'infiltration', 'or'), 15321),
 (('infiltration', 'or', 'nodule'), 15116),
 (('pulmonary', 'infiltration', '<\\s>'), 14372),
 (('active', 'pulmonary', 'infiltration'), 13986),
 (('no', 'definite', 'pulmonary'), 13707),
 (('definite', 'pulmonary', 'infiltration'), 13531),
 (('there', 'is', 'no'), 13056),
 (('<s>', 'there', 'is'), 13019),
 (('is', 'no', 'definite'), 12795),
 (('<s>', 'imp', 'no'), 12791),
 (('or', 'nodule', '<\\s>'), 12744),
 (('no', 'active', 'pulmonary'), 11724),
 (('imp', 'no', 'active'), 11133),
 (('is', 'noted', '<\\s>'), 3650),
 (('<s>', 'findings', 'no'), 2699),
 (('findings', 'no', 'detectable'), 2676),
 (('or', 'nodule', 'is'), 2127),
 (('nodule', 'is', 'noted'), 2079),
 (('fibrotic', 'infiltration', 'at'), 1799),
 (('is', 'seen', '<\\s>'), 1751)]

## Ground truth 1

Prepare Data

In [497]:
# choose reports
v1_df_1 = selectData(v1_data, 'Inspectra Lung Opacity v1', 1.0)
v1_df_1

,Reports,Report List,Inspectra Lung Opacity v1 BERT Labeler,Inspectra Lung Opacity v1 Inspectra Labeler,Inspectra Lung Opacity v1 Ground Truth,Inspectra Lung Opacity v1 Key Detect
0,CXR (PA upright)\n\nPatchy infiltration at RL...,"['<s> cxr pa upright <\\s>', '<s> patchy infil...",0,1,1.0,"<re.Match object; span=(0, 16), match='<s> pat..."
1,CXR (PA upright)\n\nReticulonodular infiltrati...,"['<s> cxr pa upright <\\s>', '<s> reticulonodu...",0,1,1.0,"<re.Match object; span=(0, 25), match='<s> ret..."
2,Chest:-\nPa chest study reveals fibronodular i...,"['<s> chest <\\s>', '<s> pa chest study reveal...",0,1,1.0,"<re.Match object; span=(0, 45), match='<s> pa ..."
3,CHEST :\nP.A. upright view .\nFibronodular inf...,"['<s> chest <\\s>', '<s> p.a <\\s>', '<s> upri...",0,1,1.0,"<re.Match object; span=(0, 22), match='<s> fib..."
4,CHEST :\nP.A. upright view .\nFibronodular inf...,"['<s> chest <\\s>', '<s> p.a <\\s>', '<s> upri...",0,1,1.0,"<re.Match object; span=(0, 22), match='<s> fib..."
...,...,...,...,...,...,...
76653,Chest PA upright \n\nknown CA breast\n\nSurgic...,"['<s> chest pa upright <\\s>', '<s> known ca b...",0,1,1.0,"<re.Match object; span=(0, 66), match='<s> fib..."
76655,CXR PA\n\nThere is interstitial infiltation at...,"['<s> cxr pa <\\s>', '<s> there is interstitia...",0,1,1.0,"<re.Match object; span=(0, 31), match='<s> the..."
76657,CXR(PA)\n \n Reticulocalcified infiltration...,"['<s> cxr pa <\\s>', '<s> reticulocalcified in...",0,1,1.0,"<re.Match object; span=(0, 27), match='<s> ret..."
76658,CXR(PA)\n\n IMP: Retioculonodular infiltratio...,"['<s> cxr pa <\\s>', '<s> imp retioculonodular...",0,1,1.0,"<re.Match object; span=(0, 30), match='<s> imp..."


In [498]:
# create report list
v1_report_1 = createReportList(v1_df_1)
v1_report_1[0]

['<s> cxr pa upright <\\s>',
 '<s> patchy infiltration at rll lul and lll <\\s>',
 '<s> mild cardiomegaly <\\s>',
 '<s> no perihilar or mediastinal mass <\\s>',
 '<s> no pleural effusion <\\s>',
 '<s> bony thorax is intact <\\s>']

Find N-grams Pattern

In [499]:
v1_bigram_1 = createPatternNgrams(v1_report_1, 2, v1_keyword)
list(v1_bigram_1.items())[:20]

[(('infiltration', 'at'), 11783),
 (('at', 'both'), 6760),
 (('reticulonodular', 'infiltration'), 6480),
 (('at', 'right'), 5184),
 (('at', 'rul'), 4846),
 (('<s>', 'reticulonodular'), 4395),
 (('both', 'lungs'), 4235),
 (('noted', '<\\s>'), 4177),
 (('is', 'seen'), 4061),
 (('is', 'noted'), 3636),
 (('unremarkable', '<\\s>'), 3511),
 (('seen', '<\\s>'), 3477),
 (('pleural', 'thickening'), 3441),
 (('<s>', 'no'), 3423),
 (('at', 'left'), 3398),
 (('opacity', 'at'), 3176),
 (('pulmonary', 'infiltration'), 3062),
 (('<s>', 'there'), 3024),
 (('<s>', 'the'), 2975),
 (('there', 'is'), 2965)]

In [500]:
v1_trigram_1 = createPatternNgrams(v1_report_1, 3, v1_keyword)
list(v1_trigram_1.items())[:20]

[(('reticulonodular', 'infiltration', 'at'), 4486),
 (('infiltration', 'at', 'both'), 3590),
 (('<s>', 'reticulonodular', 'infiltration'), 3235),
 (('infiltration', 'at', 'rul'), 2677),
 (('<s>', 'there', 'is'), 2558),
 (('is', 'noted', '<\\s>'), 2464),
 (('is', 'seen', '<\\s>'), 2343),
 (('not', 'remarkable', '<\\s>'), 2276),
 (('are', 'not', 'remarkable'), 2247),
 (('at', 'both', 'upper'), 2142),
 (('at', 'both', 'lungs'), 2136),
 (('at', 'rul', '<\\s>'), 1973),
 (('costophrenic', 'sulci', 'are'), 1733),
 (('sulci', 'are', 'not'), 1729),
 (('both', 'upper', 'lobes'), 1687),
 (('interstitial', 'infiltration', 'at'), 1680),
 (('both', 'lungs', '<\\s>'), 1612),
 (('infiltration', 'at', 'right'), 1592),
 (('nodular', 'opacity', 'at'), 1546),
 (('both', 'costophrenic', 'sulci'), 1515)]

## Ground truth -1

Prepare Data

In [501]:
# choose reports
v1_df_u = selectData(v1_data, 'Inspectra Lung Opacity v1', -1.0)
v1_df_u

,Reports,Report List,Inspectra Lung Opacity v1 BERT Labeler,Inspectra Lung Opacity v1 Inspectra Labeler,Inspectra Lung Opacity v1 Ground Truth,Inspectra Lung Opacity v1 Key Detect
6,CXR PA upright\n The chest shows mild reticul...,"['<s> cxr pa upright <\\s>', '<s> the chest sh...",0,1,-1.0,"<re.Match object; span=(20, 32), match='mild r..."
11,CXR\n Multiple nodules are noted at left lung...,"['<s> cxr <\\s>', '<s> multiple nodules are no...",0,1,-1.0,"<re.Match object; span=(52, 66), match='sugges..."
12,Chest;\n\n\n No significant change of reticul...,"['<s> chest <\\s>', '<s> no significant change...",0,1,-1.0,"<re.Match object; span=(4, 80), match='no sign..."
14,Chest film\n\nUnchanged fibrocalcific infiltra...,"['<s> chest film <\\s>', '<s> unchanged fibroc...",0,1,-1.0,"<re.Match object; span=(4, 83), match='unchang..."
31,CXR\n\nHistory of rule out TB\n\nAs compared t...,"['<s> cxr <\\s>', '<s> history of rule out tb ...",0,1,-1.0,"<re.Match object; span=(61, 195), match='no si..."
...,...,...,...,...,...,...
76641,Chest CXR\n\nNo change of a tiny nodular opaci...,"['<s> chest cxr <\\s>', '<s> no change of a ti...",0,1,-1.0,"<re.Match object; span=(4, 60), match='no chan..."
76644,CXR PA UPRIGHT\nCompared with 27/09/2017\n\n- ...,"['<s> cxr pa upright <\\s>', '<s> compared wit...",0,1,-1.0,"<re.Match object; span=(4, 50), match='no sign..."
76648,CXR PA\n\nNo change of nodular shadow at left ...,"['<s> cxr pa <\\s>', '<s> no change of nodular...",0,1,-1.0,"<re.Match object; span=(4, 81), match='no chan..."
76654,"Chest:\nNo change of emphysematous lung, RUL a...","['<s> chest <\\s>', '<s> no change of emphysem...",0,1,-1.0,"<re.Match object; span=(4, 53), match='no chan..."


In [502]:
# create report list
v1_report_u = createReportList(v1_df_u)
v1_report_u[0]

['<s> cxr pa upright <\\s>',
 '<s> the chest shows mild reticulonodular infiltration at right apical lung <\\s>',
 '<s> the left lung is clear <\\s>',
 '<s> the heart is not enlarged <\\s>',
 '<s> both costophrenic angles are clear <\\s>',
 '<s> the bony thorax is intact <\\s>']

Find N-grams Pattern

In [503]:
v1_bigram_u = createPatternNgrams(v1_report_u, 2, v1_keyword)
list(v1_bigram_u.items())[:20]

[(('change', 'of'), 20054),
 (('<s>', 'no'), 15389),
 (('infiltration', 'at'), 12524),
 (('no', 'change'), 10755),
 (('no', 'significant'), 9988),
 (('significant', 'change'), 9358),
 (('at', 'right'), 7893),
 (('<s>', 'the'), 6735),
 (('at', 'both'), 6685),
 (('lung', '<\\s>'), 5006),
 (('at', 'left'), 4955),
 (('reticulonodular', 'infiltration'), 4695),
 (('unremarkable', '<\\s>'), 4325),
 (('at', 'rul'), 4295),
 (('pleural', 'thickening'), 4069),
 (('the', 'chest'), 3923),
 (('chest', 'shows'), 3912),
 (('interstitial', 'infiltration'), 3619),
 (('noted', '<\\s>'), 3615),
 (('opacity', 'at'), 3583)]

In [504]:
v1_trigram_u = createPatternNgrams(v1_report_u, 3, v1_keyword)
list(v1_trigram_u.items())[:20]

[(('no', 'change', 'of'), 10371),
 (('no', 'significant', 'change'), 9350),
 (('significant', 'change', 'of'), 8905),
 (('<s>', 'no', 'significant'), 7279),
 (('<s>', 'no', 'change'), 5119),
 (('<s>', 'the', 'chest'), 3922),
 (('the', 'chest', 'shows'), 3912),
 (('reticulonodular', 'infiltration', 'at'), 3474),
 (('infiltration', 'at', 'both'), 3278),
 (('chest', 'shows', 'no'), 3250),
 (('shows', 'no', 'change'), 3040),
 (('<s>', 'there', 'is'), 2912),
 (('infiltration', 'at', 'right'), 2628),
 (('is', 'noted', '<\\s>'), 2447),
 (('interstitial', 'infiltration', 'at'), 2348),
 (('infiltration', 'at', 'rul'), 2315),
 (('change', 'of', 'interstitial'), 2246),
 (('there', 'is', 'no'), 2051),
 (('of', 'reticulonodular', 'infiltration'), 1996),
 (('is', 'unremarkable', '<\\s>'), 1929)]

## Unlabel

Prepare Data

In [505]:
# choose reports
v1_df = selectData(v1_data, 'Inspectra Lung Opacity v1', None)
v1_df

,Reports,Report List,Inspectra Lung Opacity v1 BERT Labeler,Inspectra Lung Opacity v1 Inspectra Labeler,Inspectra Lung Opacity v1 Ground Truth,Inspectra Lung Opacity v1 Key Detect
73,CXR PA upright\nMinimal fibroinfitlration at R...,"['<s> cxr pa upright <\\s>', '<s> minimal fibr...",0,1,NaN,NaN
307,Chest PA upright;\n \n Diffuse mild bilateral ...,"['<s> chest pa upright <\\s>', '<s> diffuse mi...",0,1,NaN,NaN
319,chest PA upright;\n\n mild diffuse bilateral p...,"['<s> chest pa upright <\\s>', '<s> mild diffu...",0,1,NaN,NaN
369,CHEST PA UPRIGHT\n\nCardiomegaly with RUL infi...,"['<s> chest pa upright <\\s>', '<s> cardiomega...",0,1,NaN,NaN
438,CXR\n\nMinimal RUL interstitial inflitration i...,"['<s> cxr <\\s>', '<s> minimal rul interstitia...",0,1,NaN,NaN
...,...,...,...,...,...,...
75421,CXR (PA)\n\nIntersttiial oapcity are seen at b...,"['<s> cxr pa <\\s>', '<s> intersttiial oapcity...",0,1,NaN,NaN
75514,CXR (PA)\n\nAtelectasis and bronchiectasis of ...,"['<s> cxr pa <\\s>', '<s> atelectasis and bron...",0,1,NaN,NaN
76013,CXR\n\nPulmonary inflitration at LLL with smal...,"['<s> cxr <\\s>', '<s> pulmonary inflitration ...",0,1,NaN,NaN
76068,"Chest:PA,upright\nCardiomegaly is noted.\nPulm...","['<s> chest pa upright <\\s>', '<s> cardiomega...",1,0,NaN,NaN


In [506]:
# create report list
v1_report = createReportList(v1_df)
v1_report[0]

['<s> cxr pa upright <\\s>',
 '<s> minimal fibroinfitlration at rll <\\s>',
 '<s> fibrosis rul <\\s>',
 '<s> prominent of both hilar regions are seen which may be lymphadenopathy or prominent pulmonary trunk <\\s>',
 '<s> normal cardiothoracic ratio <\\s>',
 '<s> artherosclerotic changes of aorta <\\s>',
 '<s> bony thorax is intact <\\s>',
 '<s> right pleural effusion is also suspected <\\s>',
 '<s> please clinical correlation <\\s>']

Find N-grams Pattern

In [507]:
v1_bigram = createPatternNgrams(v1_report, 2)
list(v1_bigram.items())[:20]

[(('<s>', 'no'), 282),
 (('pleural', 'effusion'), 218),
 (('effusion', '<\\s>'), 166),
 (('no', 'pleural'), 153),
 (('<s>', 'normal'), 130),
 (('seen', '<\\s>'), 121),
 (('noted', '<\\s>'), 114),
 (('intact', '<\\s>'), 111),
 (('is', 'noted'), 110),
 (('bony', 'thorax'), 104),
 (('<s>', 'mild'), 103),
 (('is', 'seen'), 101),
 (('size', '<\\s>'), 91),
 (('<s>', 'the'), 90),
 (('<s>', 'there'), 90),
 (('there', 'is'), 78),
 (('heart', 'size'), 78),
 (('normal', 'heart'), 76),
 (('<s>', 'bony'), 71),
 (('is', 'intact'), 71)]

In [508]:
v1_trigram = createPatternNgrams(v1_report, 3)
list(v1_trigram.items())[:20]

[(('no', 'pleural', 'effusion'), 152),
 (('<s>', 'no', 'pleural'), 151),
 (('pleural', 'effusion', '<\\s>'), 151),
 (('is', 'seen', '<\\s>'), 87),
 (('is', 'noted', '<\\s>'), 86),
 (('<s>', 'there', 'is'), 75),
 (('<s>', 'normal', 'heart'), 75),
 (('normal', 'heart', 'size'), 73),
 (('is', 'intact', '<\\s>'), 71),
 (('bony', 'thorax', 'is'), 60),
 (('thorax', 'is', 'intact'), 60),
 (('heart', 'size', '<\\s>'), 57),
 (('<s>', 'mild', 'cardiomegaly'), 53),
 (('pleural', 'effusion', 'is'), 50),
 (('<s>', 'bony', 'thorax'), 42),
 (('<s>', 'no', 'cardiomegaly'), 41),
 (('bony', 'thorax', '<\\s>'), 36),
 (('effusion', 'is', 'seen'), 35),
 (('cardiomegaly', 'is', 'noted'), 35),
 (('are', 'clear', '<\\s>'), 34)]

In [509]:
v1_fourgram = createPatternNgrams(v1_report, 4)
list(v1_fourgram.items())[:20]

[(('<s>', 'no', 'pleural', 'effusion'), 150),
 (('no', 'pleural', 'effusion', '<\\s>'), 118),
 (('<s>', 'normal', 'heart', 'size'), 73),
 (('bony', 'thorax', 'is', 'intact'), 60),
 (('thorax', 'is', 'intact', '<\\s>'), 60),
 (('normal', 'heart', 'size', '<\\s>'), 54),
 (('<s>', 'bony', 'thorax', 'is'), 41),
 (('effusion', 'is', 'seen', '<\\s>'), 35),
 (('no', 'pleural', 'effusion', 'is'), 31),
 (('angles', 'are', 'clear', '<\\s>'), 29),
 (('<s>', 'mild', 'cardiomegaly', 'is'), 29),
 (('costophrenic', 'angles', 'are', 'clear'), 28),
 (('pleural', 'effusion', 'is', 'seen'), 27),
 (('<s>', 'both', 'costophrenic', 'angles'), 26),
 (('both', 'costophrenic', 'angles', 'are'), 26),
 (('<s>', 'no', 'cardiomegaly', '<\\s>'), 22),
 (('cardiomegaly', 'is', 'noted', '<\\s>'), 22),
 (('bony', 'structures', 'are', 'intact'), 19),
 (('structures', 'are', 'intact', '<\\s>'), 19),
 (('<s>', 'normal', 'bony', 'thorax'), 19)]

In [510]:
v1_bigramk = createPatternNgrams(v1_report, 2, v1_keyword)
list(v1_bigramk.items())[:20]

[]

In [511]:
v1_trigramk = createPatternNgrams(v1_report, 3, v1_keyword)
list(v1_trigramk.items())[:20]

[]

In [512]:
v1_fourgramk = createPatternNgrams(v1_report, 4, v1_keyword)
list(v1_fourgramk.items())[:20]

[]

In [513]:
# define shorter keyword
infil_keyword_2 = r'infi|densi|opaci|'
conso_keyword_2 = 'consoli|'
opaci_keyword_2 = 'opaci|translu|air|mark|pattern|lung|reticu|scar|thicken|densi|patchy'
v1_keyword_2 = infil_keyword_2 + conso_keyword_2 + opaci_keyword_2

In [514]:
v1_bigramk = createPatternNgrams(v1_report, 2, v1_keyword_2)
list(v1_bigramk.items())[:20]

[(('infitlration', 'in'), 58),
 (('infitlration', 'at'), 52),
 (('interstitial', 'infitlration'), 41),
 (('<s>', 'there'), 38),
 (('there', 'is'), 35),
 (('noted', '<\\s>'), 26),
 (('at', 'the'), 25),
 (('fibrotic', 'infitlration'), 24),
 (('<s>', 'minimal'), 22),
 (('region', '<\\s>'), 19),
 (('minimal', 'fibrotic'), 18),
 (('seen', '<\\s>'), 14),
 (('is', 'noted'), 14),
 (('in', 'rul'), 14),
 (('in', 'both'), 14),
 (('rll', '<\\s>'), 12),
 (('<s>', 'mild'), 12),
 (('infitlration', 'is'), 12),
 (('rul', 'infitlration'), 12),
 (('of', 'the'), 12)]

In [515]:
v1_trigramk = createPatternNgrams(v1_report, 3, v1_keyword_2)
list(v1_trigramk.items())[:20]

[(('<s>', 'there', 'is'), 35),
 (('infitlration', 'at', 'the'), 23),
 (('interstitial', 'infitlration', 'at'), 21),
 (('fibrotic', 'infitlration', 'in'), 21),
 (('minimal', 'fibrotic', 'infitlration'), 17),
 (('<s>', 'minimal', 'fibrotic'), 15),
 (('is', 'noted', '<\\s>'), 14),
 (('infitlration', 'in', 'both'), 13),
 (('infitlration', 'in', 'lll'), 11),
 (('are', 'noted', '<\\s>'), 10),
 (('infitlration', 'in', 'rul'), 10),
 (('<s>', 'rul', 'infitlration'), 9),
 (('<s>', 'interstitial', 'infitlration'), 9),
 (('at', 'rll', '<\\s>'), 7),
 (('there', 'is', 'interstitial'), 7),
 (('infitlration', 'at', 'rul'), 7),
 (('infitlration', 'at', 'rll'), 7),
 (('there', 'is', 'infitlration'), 7),
 (('is', 'infitlration', 'at'), 7),
 (('interstitial', 'infitlration', 'in'), 7)]

In [516]:
v1_fourgramk = createPatternNgrams(v1_report, 4, v1_keyword_2)
list(v1_fourgramk.items())[:20]

[(('<s>', 'minimal', 'fibrotic', 'infitlration'), 15),
 (('minimal', 'fibrotic', 'infitlration', 'in'), 15),
 (('<s>', 'interstitial', 'infitlration', 'at'), 8),
 (('<s>', 'there', 'is', 'interstitial'), 7),
 (('interstitial', 'infitlration', 'at', 'the'), 7),
 (('<s>', 'there', 'is', 'infitlration'), 7),
 (('there', 'is', 'infitlration', 'at'), 7),
 (('infitlration', 'at', 'the', 'rll'), 6),
 (('is', 'infitlration', 'at', 'the'), 6),
 (('infitlration', 'in', 'lll', '<\\s>'), 6),
 (('there', 'is', 'interstitial', 'infitlration'), 5),
 (('<s>', 'there', 'is', 'improvement'), 5),
 (('there', 'is', 'improvement', 'of'), 5),
 (('in', 'both', 'upper', 'lobes'), 5),
 (('rul', 'infitlration', 'is', 'noted'), 5),
 (('infitlration', 'is', 'noted', '<\\s>'), 5),
 (('fibrotic', 'infitlration', 'in', 'rul'), 5),
 (('fibrotic', 'infitlration', 'in', 'lll'), 5),
 (('is', 'interstitial', 'infitlration', 'at'), 4),
 (('minimal', 'interstitial', 'infitlration', 'at'), 4)]

# Edema